# Model

In [64]:
using JuMP
using Gurobi
using JSON
using Dates
const MOI = JuMP.MOI

# =================================================================================
# Gurobi license
# =================================================================================

function setup_gurobi_license()
    wls_access_id = get(ENV, "WLSACCESSID", "")
    wls_secret    = get(ENV, "WLSSECRET", "")
    license_id    = get(ENV, "LICENSEID", "")
    
    if !isempty(wls_access_id) && !isempty(wls_secret)
        println("🔑 Using Gurobi license from environment variables")
        ENV["GRB_WLSACCESSID"] = wls_access_id
        ENV["GRB_WLSSECRET"]   = wls_secret
        if !isempty(license_id)
            ENV["GRB_LICENSEID"] = license_id
        end
        return
    end
    
    license_file = joinpath(pwd(), "gurobi.lic")
    if isfile(license_file)
        println("🔑 Reading Gurobi license from $license_file")
        for line in eachline(license_file)
            line = strip(line)
            if !isempty(line) && !startswith(line, "#") && occursin("=", line)
                parts = split(line, "=", limit=2)
                if length(parts) == 2
                    key = strip(parts[1])
                    val = strip(parts[2])
                    ENV[key] = val
                    ENV["GRB_" * key] = val
                end
            end
        end
        println("✅ Gurobi license configured")
    else
        println("⚠️ No Gurobi license file found")
    end
end

# =================================================================================
# INPUT PARSING
# =================================================================================

function calculate_sessions(course, num_weeks, term_config)
    ctype      = get(course, "type", "full_term")
    period_min = term_config["period_length_minutes"]
    
    if ctype == "first_half_term"
        w_start = 1
        w_end   = div(num_weeks, 2)
        req_min = 3.0 * 60       # double course
    elseif ctype == "second_half_term"
        w_start = div(num_weeks, 2) + 1
        w_end   = num_weeks
        req_min = 3.0 * 60
    else
        w_start = 1
        w_end   = num_weeks
        req_min = 1.5 * 60       # single
    end
    
    periods         = Int(ceil(req_min / period_min))
    active_weeks    = w_end - w_start + 1
    total_sessions  = active_weeks
    
    return (periods_per_session = periods,
            total_sessions      = total_sessions,
            week_start          = w_start,
            week_end            = w_end)
end

function get_lunch_periods(term_config, num_periods)
    day_start = term_config["day_start_time"]
    p_len     = term_config["period_length_minutes"]
    
    s_parts   = split(day_start, ":")
    start_min = parse(Int, s_parts[1]) * 60 + parse(Int, s_parts[2])
    
    lunch_start_min = 12 * 60
    lunch_end_min   = 12 * 60 + 30
    
    lunch_periods = Int[]
    for p in 1:num_periods
        p_start = start_min + (p-1)*p_len
        p_end   = p_start + p_len
        if max(p_start, lunch_start_min) < min(p_end, lunch_end_min)
            push!(lunch_periods, p)
        end
    end
    return lunch_periods
end

function parse_input(input_dict::Dict)
    term_config      = input_dict["term_config"]
    classrooms       = input_dict["classrooms"]
    instructors      = input_dict["instructors"]
    courses          = input_dict["courses"]
    students         = input_dict["students"]
    conflict_weights = input_dict["conflict_weights"]
    
    num_weeks = term_config["num_weeks"]
    days      = String[d for d in term_config["days"]]
    
    start_time     = term_config["day_start_time"]
    end_time       = term_config["day_end_time"]
    period_minutes = term_config["period_length_minutes"]
    
    s_parts = split(start_time, ":")
    e_parts = split(end_time,   ":")
    start_min = parse(Int, s_parts[1]) * 60 + parse(Int, s_parts[2])
    end_min   = parse(Int, e_parts[1]) * 60 + parse(Int, e_parts[2])
    day_mins  = end_min - start_min
    num_periods = div(day_mins, period_minutes)
    
    room_ids = String[r["id"] for r in classrooms]
    room_cap = Int[r["capacity"] for r in classrooms]
    
    inst_ids = String[i["id"] for i in instructors]
    
    # Availability & b2b preference
    avail = fill(true, length(instructors), length(days), num_periods)
    inst_b2b_pref = Int[get(i, "back_to_back_preference", 0) for i in instructors]  # 1 dislike, 0 neutral, -1 like
    
    for (i, inst) in enumerate(instructors)
        if haskey(inst, "availability") && !isempty(inst["availability"])
            avail[i, :, :] .= false
            for slot in inst["availability"]
                d_idx = findfirst(==(slot["day"]), days)
                p_idx = slot["period_index"] + 1
                if d_idx !== nothing && 1 <= p_idx <= num_periods
                    avail[i, d_idx, p_idx] = true
                end
            end
        end
    end
    
    # Lunch teaching preference
    inst_lunch_penalty = Float64[
        get(inst, "allow_lunch_teaching", true) ? 0.0 : 1.0
        for inst in instructors
    ]
    
    # Courses
    course_ids   = String[c["id"] for c in courses]
    course_inst  = Int[]
    course_enr   = Int[c["expected_enrollment"] for c in courses]
    for c in courses
        idx = findfirst(id -> id == c["instructor_id"], inst_ids)
        push!(course_inst, idx)
    end
    
    session_info        = [calculate_sessions(c, num_weeks, term_config) for c in courses]
    total_sessions      = [s.total_sessions for s in session_info]
    periods_per_session = [s.periods_per_session for s in session_info]
    week_starts         = [s.week_start for s in session_info]
    week_ends           = [s.week_end   for s in session_info]
    
    # Student conflicts
    students_cc = zeros(Int, length(courses), length(courses))
    c_id_map    = Dict(id => i for (i, id) in enumerate(course_ids))
    for s in students
        enrolled = [c_id_map[cid] for cid in s["enrolled_course_ids"] if haskey(c_id_map, cid)]
        for i in 1:length(enrolled), j in (i+1):length(enrolled)
            c1, c2 = enrolled[i], enrolled[j]
            students_cc[c1, c2] += 1
            students_cc[c2, c1] += 1
        end
    end
    
    w1 = Float64(get(conflict_weights, "global_student_conflict_weight",      1.0))
    w2 = Float64(get(conflict_weights, "instructor_compactness_weight",       1.0))
    w3 = Float64(get(conflict_weights, "preferred_time_slots_weight",         1.0))
    
    return (
        num_weeks          = num_weeks,
        days               = days,
        num_periods        = num_periods,
        room_ids           = room_ids,
        room_cap           = room_cap,
        inst_ids           = inst_ids,
        avail              = avail,
        inst_b2b_pref      = inst_b2b_pref,
        course_ids         = course_ids,
        course_inst        = course_inst,
        course_enr         = course_enr,
        total_sessions     = total_sessions,
        periods_per_session= periods_per_session,
        week_starts        = week_starts,
        week_ends          = week_ends,
        students_cc        = students_cc,
        w1                 = w1,
        w2                 = w2,
        w3                 = w3,
        inst_lunch_penalty = inst_lunch_penalty,
        courses            = courses,
        classrooms         = classrooms,
        instructors        = instructors,
        term_config        = term_config
    )
end

# =================================================================================
# MODEL BUILDER
# =================================================================================

function build_and_solve_model(parsed)
    C = length(parsed.course_ids)
    I = length(parsed.inst_ids)
    R = length(parsed.room_ids)
    W = parsed.num_weeks
    D = length(parsed.days)
    P = parsed.num_periods
    
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 300)
    
    println("🏗️ Building Model: $C courses, $W weeks, $(length(parsed.days)) days, $P periods")
    
    # Decision variables
    @variable(model, x[1:C, 1:W, 1:D, 1:P, 1:R], Bin)  # course start
    @variable(model, h[1:I, 1:W, 1:D, 1:P], Bin)       # instructor teaching per period
    
    # Conflict variables (S1)
    @variable(model, φ[c1 in 1:C, c2 in (c1+1):C, w in 1:W, d in 1:D, p in 1:P], Bin)
    
    # Lunch penalty (S3)
    @variable(model, π[1:C, 1:W, 1:D, 1:P], Bin)
    
    # Back-to-back structure
    # z[i,c1,c2,w,d,p] = 1 if c1 starts at p and c2 starts immediately after c1
    @variable(model, z[1:I, 1:C, 1:C, 1:W, 1:D, 1:P], Bin)
    # b2b_sess[i,c1,c2,w,d] = 1 if there exists any p with z=1 (we use c1 < c2 only)
    @variable(model, b2b_sess[1:I, 1:C, 1:C, 1:W, 1:D], Bin)
    # teach_c[i,c,w,d] = 1 if instructor i teaches course c that day
    @variable(model, teach_c[1:I, 1:C, 1:W, 1:D], Bin)
    # has_teaching[i,w,d] = 1 if instructor i teaches at least one course on day (w,d)
    @variable(model, has_teaching[1:I, 1:W, 1:D], Bin)
    
    # Helper: valid start periods for course c that occupy period p
    function valid_starts(c, current_p)
        dur = parsed.periods_per_session[c]
        return max(1, current_p - dur + 1):min(current_p, P)
    end
    
    # --------------------------- Hard constraints ---------------------------
    
    # C1: Teacher conflict
    for i in 1:I, w in 1:W, d in 1:D, p in 1:P
        my_courses = [c for c in 1:C if parsed.course_inst[c] == i]
        occ = @expression(model,
            sum(x[c,w,d,s,r]
                for c in my_courses, r in 1:R, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= 1)
        @constraint(model, h[i,w,d,p] == occ)
    end
    
    # C2: Classroom conflict
    for r in 1:R, w in 1:W, d in 1:D, p in 1:P
        @constraint(model,
            sum(x[c,w,d,s,r]
                for c in 1:C, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P) <= 1
        )
    end
    
    # C3: required sessions, duration & active weeks
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        dur     = parsed.periods_per_session[c]
        req_s   = parsed.total_sessions[c]
        
        @constraint(model,
            sum(x[c,w,d,p,r]
                for w in w_start:w_end, d in 1:D, p in 1:P, r in 1:R
                if p + dur - 1 <= P) == req_s
        )
        
        all_weeks      = 1:W
        inactive_weeks = setdiff(all_weeks, w_start:w_end)
        for w in inactive_weeks
            @constraint(model,
                sum(x[c,w,d,p,r] for d in 1:D, p in 1:P, r in 1:R) == 0
            )
        end
    end
    
    # C4: availability
    for c in 1:C
        inst = parsed.course_inst[c]
        dur  = parsed.periods_per_session[c]
        for w in 1:W, d in 1:D, p in 1:P, r in 1:R
            if p + dur - 1 <= P
                is_avail = all(parsed.avail[inst,d,t] for t in p:(p+dur-1))
                if !is_avail
                    @constraint(model, x[c,w,d,p,r] == 0)
                end
            else
                @constraint(model, x[c,w,d,p,r] == 0)
            end
        end
    end
    
    # C7: capacity
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        @constraint(model, parsed.course_enr[c] * x[c,w,d,p,r] <= parsed.room_cap[r])
    end
    
    # C8: at most one session per day per course
    for c in 1:C, w in 1:W, d in 1:D
        dur = parsed.periods_per_session[c]
        @constraint(model,
            sum(x[c,w,d,p,r] for p in 1:P, r in 1:R if p + dur - 1 <= P) <= 1
        )
    end
    
    # C9: pattern consistency across weeks
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        if w_end > w_start
            for w in (w_start+1):w_end, d in 1:D, p in 1:P, r in 1:R
                @constraint(model, x[c,w,d,p,r] == x[c,w_start,d,p,r])
            end
        end
    end
    
    # --------------------------- Soft constraints ---------------------------
    
    # S1: student conflicts
    for c1 in 1:C, c2 in (c1+1):C
        if parsed.students_cc[c1,c2] > 0
            for w in 1:W, d in 1:D, p in 1:P
                occ1 = @expression(model,
                    sum(x[c1,w,d,s,r] for r in 1:R, s in valid_starts(c1,p)
                        if s + parsed.periods_per_session[c1] - 1 <= P)
                )
                occ2 = @expression(model,
                    sum(x[c2,w,d,s,r] for r in 1:R, s in valid_starts(c2,p)
                        if s + parsed.periods_per_session[c2] - 1 <= P)
                )
                @constraint(model, occ1 + occ2 <= 1 + φ[c1,c2,w,d,p])
            end
        end
    end
    
    # ---------- Back-to-back structure (S2) ----------
    
    # z & b2b_sess for unordered pairs c1 < c2
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c1,w,d,p,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c2,w,d,p+len1,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] >=
                            sum(x[c1,w,d,p,r]      for r in 1:R) +
                            sum(x[c2,w,d,p+len1,r] for r in 1:R) - 1
                    )
                end
            end
        end
    end
    
    # b2b_sess is OR of z over p
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                @constraint(model,
                    b2b_sess[i,c1,c2,w,d] <=
                        sum(z[i,c1,c2,w,d,p] for p in 1:(P-len1))
                )
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= b2b_sess[i,c1,c2,w,d]
                    )
                end
            end
        end
    end
    
    # teach_c: whether instructor teaches course c that day
    for i in 1:I, c in 1:C, w in 1:W, d in 1:D
        if parsed.course_inst[c] != i
            @constraint(model, teach_c[i,c,w,d] == 0)
        else
            total = @expression(model,
                sum(x[c,w,d,p,r] for p in 1:P, r in 1:R)
            )
            @constraint(model, teach_c[i,c,w,d] <= total)
            @constraint(model, teach_c[i,c,w,d] >= total / P)
        end
    end
    
    # has_teaching[i,w,d] = 1 if instructor i teaches at least one course on day (w,d)
    for i in 1:I, w in 1:W, d in 1:D
        T_sum = @expression(model,
            sum(teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
        )
        max_T = sum(1 for c in 1:C if parsed.course_inst[c] == i)
        if max_T > 0
            @constraint(model, has_teaching[i,w,d] <= T_sum)
            @constraint(model, has_teaching[i,w,d] >= T_sum / max_T)
        else
            @constraint(model, has_teaching[i,w,d] == 0)
        end
    end
    
    # S3: lunch periods
    lunch_periods = get_lunch_periods(parsed.term_config, P)
    for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods
        occ = @expression(model,
            sum(x[c,w,d,s,r] for r in 1:R, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= π[c,w,d,p])
    end
    
    # --------------------------- Objective ---------------------------
    
    # S1: student conflicts
    obj_s1 = @expression(model,
        sum(parsed.w1 * parsed.students_cc[c1,c2] * φ[c1,c2,w,d,p]
            for c1 in 1:C, c2 in (c1+1):C, w in 1:W, d in 1:D, p in 1:P
            if parsed.students_cc[c1,c2] > 0)
    )
    
     # S2: symmetric back-to-back preference (per instructor/day)
    # Only calculate for days where instructor teaches (T > 0)

    # Use a vector of QuadExpr, because terms are bilinear
    obj_s2_terms = JuMP.QuadExpr[]

    for i in 1:I, w in 1:W, d in 1:D
        pref = parsed.inst_b2b_pref[i]
        if pref == 0
            continue
        end

        # Tᵢwd = #courses taught that day
        T_expr = @expression(model,
            sum(teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
        )

        # Bᵢwd = #b2b unordered pairs that day
        B_expr = @expression(model,
            sum(b2b_sess[i,c1,c2,w,d]
                for c1 in 1:(C-1), c2 in (c1+1):C
                if parsed.course_inst[c1] == i && parsed.course_inst[c2] == i)
        )

        # One S2 contribution term (this is quadratic)
        term = parsed.w2 * pref * has_teaching[i,w,d] * (2 * B_expr - (T_expr - 1))

        push!(obj_s2_terms, term)
    end

    # If no instructor has a nonzero pref, S2 = 0
    obj_s2 = isempty(obj_s2_terms) ? 0.0 : sum(obj_s2_terms)

    # S3: lunch penalty
    obj_s3 = @expression(model,
        sum(parsed.w3 *
            parsed.inst_lunch_penalty[parsed.course_inst[c]] *
            π[c,w,d,p]
            for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods)
    )
    
    @objective(model, Min, obj_s1 + obj_s2 + obj_s3)
    
    println("  Constraints: ",
        num_constraints(model, AffExpr, MOI.LessThan{Float64}) +
        num_constraints(model, AffExpr, MOI.EqualTo{Float64}))
    println("  Variables:   ", num_variables(model))
    println("  🚀 Optimizing...")
    
    optimize!(model)
    
    status = termination_status(model)
    println("  Status:     $status")
    println("  Solve time: $(round(solve_time(model), digits=3))s")
    
    println("=== Objective decomposition ===")
    println("  S1_student_conflicts = ", value(obj_s1))
    println("  S2_back_to_back      = ", value(obj_s2))
    println("  S3_lunch_penalty     = ", value(obj_s3))
    println("  Total objective      = ", objective_value(model))
    
    # ---------- DEBUG: S2 details ----------
    println("=== S2 DEBUG BY INSTRUCTOR / DAY ===")
    vals_teach_c   = value.(teach_c)
    vals_b2b_sess  = value.(b2b_sess)
    for i in 1:I
        pref = parsed.inst_b2b_pref[i]
        if pref == 0
            continue
        end
        println("Instructor $(parsed.inst_ids[i]) pref = $pref (−1 likes b2b, +1 dislikes b2b)")
        for w in 1:W, d in 1:D
            # T = #courses taught that day
            T = sum(vals_teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
            if T < 0.5
                continue
            end
            B = 0.0
            for c1 in 1:(C-1), c2 in (c1+1):C
                if parsed.course_inst[c1] == i && parsed.course_inst[c2] == i
                    B += vals_b2b_sess[i,c1,c2,w,d]
                end
            end
            E = T - 1.0
            contrib = parsed.w2 * pref * (2B - (T - 1.0))
            println("  Week $w, Day $(parsed.days[d]): T=$T, edges(E)=T-1=$E, B=$B, contrib=$contrib")
        end
    end
    
    println("=== Assignments ===")
    vals_x = value.(x)
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        if vals_x[c,w,d,p,r] > 0.5
            println("Course $(parsed.course_ids[c]) in room $(parsed.room_ids[r])",
                    " | week=$w day=$(parsed.days[d]) period_start=$p",
                    " (len=$(parsed.periods_per_session[c]))")
        end
    end
    
    return model, x
end

# =================================================================================
# OUTPUT FORMATTERS
# =================================================================================

function format_optimal_solution(model, x, parsed)
    assignments = []
    vals = value.(x)
    C,W,D,P,R = size(vals)
    
    session_counters = Dict{Int,Int}()
    for c in 1:C
        session_counters[c] = 0
    end
    
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        if vals[c,w,d,p,r] > 0.5
            session_counters[c] += 1
            snum = session_counters[c]
            course = parsed.courses[c]
            room   = parsed.classrooms[r]
            inst_idx = parsed.course_inst[c]
            inst     = parsed.instructors[inst_idx]
            push!(assignments, Dict(
                "course_id"         => parsed.course_ids[c],
                "course_session_id" => "$(parsed.course_ids[c])_S$(snum)",
                "session_number"    => snum,
                "course_name"       => get(course, "name", ""),
                "room_id"           => parsed.room_ids[r],
                "room_name"         => get(room, "name", ""),
                "week"              => w - 1,
                "day"               => parsed.days[d],
                "period_start"      => p - 1,
                "period_length"     => parsed.periods_per_session[c],
                "instructor_id"     => get(inst, "id", ""),
                "instructor_name"   => get(inst, "name", "")
            ))
        end
    end
    
    obj_val = has_values(model) ? objective_value(model) : nothing
    
    return Dict(
        "status"             => termination_status(model) == MOI.OPTIMAL ? "optimal" : "time_limit",
        "objective_value"    => obj_val,
        "solve_time_seconds" => solve_time(model),
        "hard_constraints_ok"=> true,
        "violated_hard_constraints" => String[],
        "soft_constraint_summary" => Dict(
            "S1_student_conflicts" => Dict(
                "weighted_penalty" => obj_val,
                "weight" => parsed.w1,
                "details"=> "Student schedule conflicts (weight: $(parsed.w1))"
            ),
            "S2_instructor_compactness" => Dict(
                "weighted_penalty" => 0.0,
                "weight" => parsed.w2,
                "details"=> "Instructor back-to-back preference (weight: $(parsed.w2))"
            ),
            "S3_preferred_time_slots" => Dict(
                "weighted_penalty" => 0.0,
                "weight" => parsed.w3,
                "details"=> "Lunch period penalties (weight: $(parsed.w3))"
            )
        ),
        "schedule" => Dict("assignments" => assignments),
        "diagnostics" => Dict(
            "room_capacity_violations" => [],
            "teacher_overlaps"         => [],
            "student_conflicts"        => [],
            "lunch_violations"         => []
        ),
        "metadata" => Dict(
            "solver"                     => "Gurobi (Julia/JuMP)",
            "constraint_metadata_version"=> "1.0",
            "num_assignments"            => length(assignments)
        )
    )
end

function format_infeasible_solution(model, parsed)
    try
        compute_conflict!(model)
        println("  IIS computation attempted")
    catch
        println("  IIS computation failed")
    end
    return Dict(
        "status"             => "infeasible",
        "objective_value"    => nothing,
        "solve_time_seconds" => solve_time(model),
        "hard_constraints_ok"=> false,
        "violated_hard_constraints" => ["multiple_constraints"],
        "soft_constraint_summary"   => Dict(),
        "schedule" => Dict("assignments" => []),
        "diagnostics" => Dict(
            "infeasibility_explanation" => "No feasible schedule satisfies all hard constraints"
        ),
        "metadata" => Dict(
            "solver"                     => "Gurobi (Julia/JuMP)",
            "constraint_metadata_version"=> "1.0"
        )
    )
end

function format_output(model, x, parsed)
    status = termination_status(model)
    if status in (MOI.INFEASIBLE, MOI.INFEASIBLE_OR_UNBOUNDED)
        return format_infeasible_solution(model, parsed)
    elseif status != MOI.OPTIMAL && status != MOI.TIME_LIMIT
        return Dict(
            "status"             => string(status),
            "objective_value"    => nothing,
            "solve_time_seconds" => solve_time(model),
            "hard_constraints_ok"=> false,
            "violated_hard_constraints" => [],
            "soft_constraint_summary"   => Dict(),
            "schedule" => Dict("assignments" => []),
            "diagnostics" => Dict(
                "gurobi_status" => string(status),
                "message"       => "Solver terminated with status: $(string(status))"
            ),
            "metadata" => Dict(
                "solver" => "Gurobi (Julia/JuMP)"
            )
        )
    else
        return format_optimal_solution(model, x, parsed)
    end
end

# =================================================================================
# TOP-LEVEL SOLVER
# =================================================================================

function solve_scheduling_problem(input_dict::Dict)
    try
        println("📊 Julia solver started at $(now())")
        setup_gurobi_license()
        parsed        = parse_input(input_dict)
        model, x      = build_and_solve_model(parsed)
        output        = format_output(model, x, parsed)
        println("✅ Julia solver completed: $(output["status"])")
        return output
    catch e
        msg = string(e)
        println("❌ Julia solver error: $msg")
        tb = try
            sprint(showerror, e, catch_backtrace())
        catch
            "Stack trace unavailable"
        end
        println("   Traceback: $tb")
        return Dict(
            "status"             => "error",
            "objective_value"    => nothing,
            "solve_time_seconds" => 0.0,
            "hard_constraints_ok"=> false,
            "violated_hard_constraints" => [],
            "soft_constraint_summary"   => Dict(),
            "schedule" => Dict("assignments" => []),
            "diagnostics" => Dict(
                "error"     => msg,
                "traceback" => tb
            ),
            "metadata" => Dict(
                "solver"   => "Gurobi (Julia/JuMP)",
                "timestamp"=> string(now())
            )
        )
    end
end

solve_scheduling_problem (generic function with 1 method)

# Implementation Test

### Test Functions

In [35]:
# ===========================
# CONSTRAINT TEST HELPERS
# ===========================

"""
Extract assignments vector from solver output.
"""
function get_assignments(output::Dict)
    return output["schedule"]["assignments"]::Vector{Any}
end

"""
Return (course_idx, inst_idx, room_idx) lookup Dicts.
"""
function build_index_maps(parsed)
    course_idx = Dict{String,Int}(cid => i for (i, cid) in enumerate(parsed.course_ids))
    inst_idx   = Dict{String,Int}(iid => i for (i, iid) in enumerate(parsed.inst_ids))
    room_idx   = Dict{String,Int}(rid => i for (i, rid) in enumerate(parsed.room_ids))
    return (course_idx = course_idx, inst_idx = inst_idx, room_idx = room_idx)
end

"""
Given an assignment Dict and index maps, get (c_idx, i_idx, r_idx).
"""
function assignment_indices(a::Dict{String,Any}, idxs, parsed)
    c_idx = idxs.course_idx[a["course_id"]]
    i_idx = idxs.inst_idx[a["instructor_id"]]
    r_idx = idxs.room_idx[a["room_id"]]
    return c_idx, i_idx, r_idx
end


assignment_indices

In [57]:
"""
C1: Teacher conflict
Verify that no instructor teaches overlapping sessions in the returned schedule.
"""
function test_C1_teacher_conflict(input_dict::Dict)
    parsed  = parse_input(input_dict)
    output  = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns   = get_assignments(output)

    P    = parsed.num_periods
    days = parsed.days

    # Map: (inst_id, week, day) -> occupancy vector over periods
    by_iwd = Dict{Tuple{String,Int,String}, Vector{Int}}()

    for a_any in assns
        a = a_any::Dict{String,Any}
        inst_id = String(a["instructor_id"])
        week0   = Int(a["week"])           # 0-based
        day_str = String(a["day"])
        p0      = Int(a["period_start"])   # 0-based
        len     = Int(a["period_length"])

        key = (inst_id, week0, day_str)
        occ = get!(by_iwd, key, fill(0, P))
        for t in p0:(p0+len-1)
            occ[t+1] += 1
        end
    end

    for ((inst_id, week0, day_str), occ) in by_iwd
        @assert all(v <= 1 for v in occ) "C1 violated: instructor $inst_id has overlapping sessions on week=$week0 day=$day_str"
    end

    println("✅ C1 teacher conflict test passed")
end

"""
C2: Classroom conflict
Verify that no room hosts overlapping sessions.
"""
function test_C2_classroom_conflict(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    P = parsed.num_periods

    # Map: (room_id, week, day) -> occupancy vector
    by_rwd = Dict{Tuple{String,Int,String}, Vector{Int}}()

    for a_any in assns
        a       = a_any::Dict{String,Any}
        room_id = String(a["room_id"])
        week0   = Int(a["week"])
        day_str = String(a["day"])
        p0      = Int(a["period_start"])
        len     = Int(a["period_length"])

        key = (room_id, week0, day_str)
        occ = get!(by_rwd, key, fill(0, P))
        for t in p0:(p0+len-1)
            occ[t+1] += 1
        end
    end

    for ((room_id, week0, day_str), occ) in by_rwd
        @assert all(v <= 1 for v in occ) "C2 violated: room $room_id has overlapping sessions on week=$week0 day=$day_str"
    end

    println("✅ C2 classroom conflict test passed")
end

"""
C3: required sessions, duration & active weeks

Check that each course:
  - Has exactly total_sessions[c] assigned sessions
  - Only uses weeks in [week_start, week_end]
"""
function test_C3_required_sessions_and_weeks(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    C    = length(parsed.course_ids)

    # Count sessions per course, track weeks used
    sessions_per_course = [0 for _ in 1:C]
    weeks_used          = [Set{Int}() for _ in 1:C]

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, _ = assignment_indices(a, idxs, parsed)
        sessions_per_course[c_idx] += 1
        push!(weeks_used[c_idx], Int(a["week"]) + 1)  # back to 1-based weeks
    end

    for c in 1:C
        req_s   = parsed.total_sessions[c]
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]

        @assert sessions_per_course[c] == req_s "C3 violated: course $(parsed.course_ids[c]) has $(sessions_per_course[c]) sessions, expected $req_s"

        for w in weeks_used[c]
            @assert w_start <= w <= w_end "C3 violated: course $(parsed.course_ids[c]) scheduled in inactive week $w (allowed $w_start-$w_end)"
        end
    end

    println("✅ C3 required sessions & active weeks test passed")
end

"""
C4: instructor availability
Verify that every assigned slot respects parsed.avail for the instructor.
"""
function test_C4_instructor_availability(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    days = parsed.days
    P    = parsed.num_periods

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, i_idx, _ = assignment_indices(a, idxs, parsed)

        day_str = String(a["day"])
        d_idx   = findfirst(==(day_str), days)
        @assert d_idx !== nothing "Unknown day in assignment: $day_str"

        p0  = Int(a["period_start"]) + 1  # to 1-based
        len = Int(a["period_length"])

        for t in p0:(p0+len-1)
            @assert 1 <= t <= P
            @assert parsed.avail[i_idx, d_idx, t] "C4 violated: instructor $(parsed.inst_ids[i_idx]) unavailable at day=$day_str, period=$t"
        end
    end

    println("✅ C4 instructor availability test passed")
end
"""
C7: classroom capacity
Verify that assigned course enrollment never exceeds room capacity.
"""
function test_C7_classroom_capacity(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, r_idx = assignment_indices(a, idxs, parsed)

        enr = parsed.course_enr[c_idx]
        cap = parsed.room_cap[r_idx]

        @assert enr <= cap "C7 violated: course $(parsed.course_ids[c_idx]) enrollment=$enr > room $(parsed.room_ids[r_idx]) capacity=$cap"
    end

    println("✅ C7 classroom capacity test passed")
end

"""
C8: at most one session per day per course
Verify that each course has at most one session on any given (week, day).
"""
function test_C8_one_session_per_course_per_day(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    # Map: (course_id, week, day) -> count
    by_cwd = Dict{Tuple{String,Int,String}, Int}()

    for a_any in assns
        a        = a_any::Dict{String,Any}
        cid      = String(a["course_id"])
        week0    = Int(a["week"])
        day_str  = String(a["day"])
        key      = (cid, week0, day_str)
        by_cwd[key] = get(by_cwd, key, 0) + 1
    end

    for ((cid, week0, day_str), cnt) in by_cwd
        @assert cnt <= 1 "C8 violated: course $cid has $cnt sessions on week=$week0 day=$day_str"
    end

    println("✅ C8 one-session-per-course-per-day test passed")
end

"""
C9: weekly pattern consistency
For courses with week_end > week_start, verify that:
  - every active week has exactly one session, and
  - (day, period_start, room_id) is identical across those weeks.
"""
function test_C9_weekly_pattern_consistency(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    C    = length(parsed.course_ids)

    # Group assignments by (course_idx, week)
    by_cw = Dict{Tuple{Int,Int}, Dict{String,Any}}()
    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, _ = assignment_indices(a, idxs, parsed)
        week1 = Int(a["week"]) + 1  # to 1-based for comparison to week_starts/ends
        by_cw[(c_idx, week1)] = a
    end

    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        if w_end > w_start
            # baseline pattern = first active week
            base_a = get(by_cw, (c, w_start), nothing)
            @assert base_a !== nothing "C9 violated: course $(parsed.course_ids[c]) missing assignment in week $w_start"

            base_day   = String(base_a["day"])
            base_p0    = Int(base_a["period_start"])
            base_room  = String(base_a["room_id"])

            for w in (w_start+1):w_end
                a = get(by_cw, (c, w), nothing)
                @assert a !== nothing "C9 violated: course $(parsed.course_ids[c]) missing assignment in week $w"

                day   = String(a["day"])
                p0    = Int(a["period_start"])
                room  = String(a["room_id"])

                @assert day == base_day "C9 violated: course $(parsed.course_ids[c]) day differs in week $w ($day vs $base_day)"
                @assert p0 == base_p0   "C9 violated: course $(parsed.course_ids[c]) period_start differs in week $w ($p0 vs $base_p0)"
                @assert room == base_room "C9 violated: course $(parsed.course_ids[c]) room differs in week $w ($room vs $base_room)"
            end
        end
    end

    println("✅ C9 weekly pattern consistency test passed")
end


test_C9_weekly_pattern_consistency

In [65]:
#############################
# TEST IMPORTS & HELPERS   #
#############################

using JuMP
using Gurobi
using JSON
using Dates
using Base: deepcopy

# Assume the following are already defined above this cell:
#   parse_input(input_dict::Dict)
#   solve_scheduling_problem(input_dict::Dict)
#   get_lunch_periods(term_config, num_periods)

"""
Extract assignments vector from solver output Dict.
"""
function get_assignments(output::Dict)
    return output["schedule"]["assignments"]::Vector{Any}
end

"""
Build quick lookup maps from IDs to indices in `parsed`.
"""
function build_index_maps(parsed)
    course_idx = Dict{String,Int}(cid => i for (i, cid) in enumerate(parsed.course_ids))
    inst_idx   = Dict{String,Int}(iid => i for (i, iid) in enumerate(parsed.inst_ids))
    room_idx   = Dict{String,Int}(rid => i for (i, rid) in enumerate(parsed.room_ids))
    return (course = course_idx, inst = inst_idx, room = room_idx)
end

"""
Given a single assignment Dict, return (course_index, inst_index, room_index).
"""
function assignment_indices(a::Dict{String,Any}, idxs, parsed)
    cid  = String(a["course_id"])
    rid  = String(a["room_id"])
    iid  = String(a["instructor_id"])
    c_idx = idxs.course[cid]
    r_idx = idxs.room[rid]
    i_idx = idxs.inst[iid]
    return c_idx, i_idx, r_idx
end

#############################
# HARD CONSTRAINT TESTS     #
#############################

"""
C1: Teacher conflict
Verify that no instructor teaches overlapping sessions in the returned schedule.
"""
function test_C1_teacher_conflict(input_dict::Dict)
    parsed  = parse_input(input_dict)
    output  = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns   = get_assignments(output)

    P = parsed.num_periods

    # Map: (inst_id, week, day) -> occupancy vector over periods
    by_iwd = Dict{Tuple{String,Int,String}, Vector{Int}}()

    for a_any in assns
        a = a_any::Dict{String,Any}
        inst_id = String(a["instructor_id"])
        week0   = Int(a["week"])           # 0-based
        day_str = String(a["day"])
        p0      = Int(a["period_start"])   # 0-based
        len     = Int(a["period_length"])

        key = (inst_id, week0, day_str)
        occ = get!(by_iwd, key, fill(0, P))
        for t in p0:(p0+len-1)
            occ[t+1] += 1
        end
    end

    for ((inst_id, week0, day_str), occ) in by_iwd
        @assert all(v <= 1 for v in occ) "C1 violated: instructor $inst_id has overlapping sessions on week=$week0 day=$day_str"
    end

    println("✅ C1 teacher conflict test passed")
end

"""
C2: Classroom conflict
Verify that no room hosts overlapping sessions.
"""
function test_C2_classroom_conflict(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    P = parsed.num_periods

    # Map: (room_id, week, day) -> occupancy vector
    by_rwd = Dict{Tuple{String,Int,String}, Vector{Int}}()

    for a_any in assns
        a       = a_any::Dict{String,Any}
        room_id = String(a["room_id"])
        week0   = Int(a["week"])
        day_str = String(a["day"])
        p0      = Int(a["period_start"])
        len     = Int(a["period_length"])

        key = (room_id, week0, day_str)
        occ = get!(by_rwd, key, fill(0, P))
        for t in p0:(p0+len-1)
            occ[t+1] += 1
        end
    end

    for ((room_id, week0, day_str), occ) in by_rwd
        @assert all(v <= 1 for v in occ) "C2 violated: room $room_id has overlapping sessions on week=$week0 day=$day_str"
    end

    println("✅ C2 classroom conflict test passed")
end

"""
C3: required sessions, duration & active weeks

Check that each course:
  - Has exactly total_sessions[c] assigned sessions
  - Only uses weeks in [week_start, week_end]
"""
function test_C3_required_sessions_and_weeks(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    C    = length(parsed.course_ids)

    sessions_per_course = [0 for _ in 1:C]
    weeks_used          = [Set{Int}() for _ in 1:C]

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, _ = assignment_indices(a, idxs, parsed)
        sessions_per_course[c_idx] += 1
        push!(weeks_used[c_idx], Int(a["week"]) + 1)  # back to 1-based weeks
    end

    for c in 1:C
        req_s   = parsed.total_sessions[c]
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]

        @assert sessions_per_course[c] == req_s "C3 violated: course $(parsed.course_ids[c]) has $(sessions_per_course[c]) sessions, expected $req_s"

        for w in weeks_used[c]
            @assert w_start <= w <= w_end "C3 violated: course $(parsed.course_ids[c]) scheduled in inactive week $w (allowed $w_start-$w_end)"
        end
    end

    println("✅ C3 required sessions & active weeks test passed")
end

"""
C4: instructor availability
Verify that every assigned slot respects parsed.avail for the instructor.
"""
function test_C4_instructor_availability(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    days = parsed.days
    P    = parsed.num_periods

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, i_idx, _ = assignment_indices(a, idxs, parsed)

        day_str = String(a["day"])
        d_idx   = findfirst(==(day_str), days)
        @assert d_idx !== nothing "Unknown day in assignment: $day_str"

        p0  = Int(a["period_start"]) + 1  # to 1-based
        len = Int(a["period_length"])

        for t in p0:(p0+len-1)
            @assert 1 <= t <= P
            @assert parsed.avail[i_idx, d_idx, t] "C4 violated: instructor $(parsed.inst_ids[i_idx]) unavailable at day=$day_str, period=$t"
        end
    end

    println("✅ C4 instructor availability test passed")
end

"""
C7: classroom capacity
Verify that assigned course enrollment never exceeds room capacity.
"""
function test_C7_classroom_capacity(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)

    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, r_idx = assignment_indices(a, idxs, parsed)

        enr = parsed.course_enr[c_idx]
        cap = parsed.room_cap[r_idx]

        @assert enr <= cap "C7 violated: course $(parsed.course_ids[c_idx]) enrollment=$enr > room $(parsed.room_ids[r_idx]) capacity=$cap"
    end

    println("✅ C7 classroom capacity test passed")
end

"""
C8: at most one session per day per course
Verify that each course has at most one session on any given (week, day).
"""
function test_C8_one_session_per_course_per_day(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    # Map: (course_id, week, day) -> count
    by_cwd = Dict{Tuple{String,Int,String}, Int}()

    for a_any in assns
        a        = a_any::Dict{String,Any}
        cid      = String(a["course_id"])
        week0    = Int(a["week"])
        day_str  = String(a["day"])
        key      = (cid, week0, day_str)
        by_cwd[key] = get(by_cwd, key, 0) + 1
    end

    for ((cid, week0, day_str), cnt) in by_cwd
        @assert cnt <= 1 "C8 violated: course $cid has $cnt sessions on week=$week0 day=$day_str"
    end

    println("✅ C8 one-session-per-course-per-day test passed")
end

"""
C9: weekly pattern consistency
For courses with week_end > week_start, verify that:
  - every active week has exactly one session, and
  - (day, period_start, room_id) is identical across those weeks.
"""
function test_C9_weekly_pattern_consistency(input_dict::Dict)
    parsed = parse_input(input_dict)
    output = solve_scheduling_problem(input_dict)
    @assert output["status"] in ("optimal", "time_limit")
    assns  = get_assignments(output)

    idxs = build_index_maps(parsed)
    C    = length(parsed.course_ids)

    # Group assignments by (course_idx, week)
    by_cw = Dict{Tuple{Int,Int}, Dict{String,Any}}()
    for a_any in assns
        a = a_any::Dict{String,Any}
        c_idx, _, _ = assignment_indices(a, idxs, parsed)
        week1 = Int(a["week"]) + 1  # to 1-based for comparison to week_starts/ends
        by_cw[(c_idx, week1)] = a
    end

    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        if w_end > w_start
            base_a = get(by_cw, (c, w_start), nothing)
            @assert base_a !== nothing "C9 violated: course $(parsed.course_ids[c]) missing assignment in week $w_start"

            base_day   = String(base_a["day"])
            base_p0    = Int(base_a["period_start"])
            base_room  = String(base_a["room_id"])

            for w in (w_start+1):w_end
                a = get(by_cw, (c, w), nothing)
                @assert a !== nothing "C9 violated: course $(parsed.course_ids[c]) missing assignment in week $w"

                day   = String(a["day"])
                p0    = Int(a["period_start"])
                room  = String(a["room_id"])

                @assert day  == base_day   "C9 violated: course $(parsed.course_ids[c]) day differs in week $w ($day vs $base_day)"
                @assert p0   == base_p0    "C9 violated: course $(parsed.course_ids[c]) period_start differs in week $w ($p0 vs $base_p0)"
                @assert room == base_room  "C9 violated: course $(parsed.course_ids[c]) room differs in week $w ($room vs $base_room)"
            end
        end
    end

    println("✅ C9 weekly pattern consistency test passed")
end

#############################
# SOFT CONSTRAINT HELPERS   #
#############################

"""
Compute a "student conflict score" from assignments and parsed data.

For each (week, day, period), if two courses overlap in that period,
we add students_cc[c1,c2] once for that period.
"""
function compute_student_conflicts(assignments::Vector{Any}, parsed)
    P    = parsed.num_periods

    course_idx = Dict{String,Int}(cid => i for (i, cid) in enumerate(parsed.course_ids))

    # grid[(week0, day_str, period0)] = Vector{Int} of course indices
    grid = Dict{Tuple{Int,String,Int}, Vector{Int}}()

    for a_any in assignments
        a       = a_any::Dict{String,Any}
        cid     = String(a["course_id"])
        c_idx   = course_idx[cid]
        week0   = Int(a["week"])
        day_str = String(a["day"])
        p0      = Int(a["period_start"])
        len     = Int(a["period_length"])

        for t0 in p0:(p0+len-1)
            key = (week0, day_str, t0)
            v   = get!(grid, key, Int[])
            push!(v, c_idx)
        end
    end

    total_conflicts = 0
    students_cc = parsed.students_cc

    for (_, courses_here) in grid
        if length(courses_here) >= 2
            for i in 1:length(courses_here)-1
                for j in (i+1):length(courses_here)
                    c1 = courses_here[i]
                    c2 = courses_here[j]
                    if c1 != c2 && students_cc[c1,c2] > 0
                        total_conflicts += students_cc[c1,c2]
                    end
                end
            end
        end
    end

    return total_conflicts
end

"""
Compute total "lunch penalty units" from a schedule:

For instructors with inst_lunch_penalty > 0 (i.e., they don't want lunch teaching),
count how many course-period-assignments intersect any lunch period.
"""
function compute_lunch_penalty(assignments::Vector{Any}, parsed)
    lunch_periods = Set(get_lunch_periods(parsed.term_config, parsed.num_periods))
    inst_idx = Dict{String,Int}(iid => i for (i, iid) in enumerate(parsed.inst_ids))

    total_lunch_hits = 0

    for a_any in assignments
        a         = a_any::Dict{String,Any}
        inst_id   = String(a["instructor_id"])
        i_idx     = inst_idx[inst_id]
        penalty_w = parsed.inst_lunch_penalty[i_idx]

        if penalty_w <= 0.0
            continue
        end

        p0  = Int(a["period_start"]) + 1  # 1-based
        len = Int(a["period_length"])

        for t in p0:(p0+len-1)
            if t in lunch_periods
                total_lunch_hits += 1
            end
        end
    end

    return total_lunch_hits
end

"""
Compute back-to-back metrics for each (instructor, week, day) in a way that
matches the MILP's S2 structure.

Returns:
  Dict{ (inst_id, week0, day_str) => (T, B) }

Where:
  - T = total number of sessions that instructor teaches on that (week, day)
  - B = number of "b2b edges" according to the same (c1,c2,len1) logic
        used in the S2 constraints:
            c1 < c2
            p2 == p1 + periods_per_session[c1]
"""
function compute_b2b_metrics(assignments::Vector{Any}, parsed)
    idxs = build_index_maps(parsed)

    # key = (inst_id, week0, day_str)
    # value = Dict{course_index => Vector{start_period0}}
    sessions_by_key_and_course =
        Dict{Tuple{String,Int,String}, Dict{Int,Vector{Int}}}()

    for a_any in assignments
        a       = a_any::Dict{String,Any}
        inst_id = String(a["instructor_id"])
        week0   = Int(a["week"])          # 0-based
        day_str = String(a["day"])
        cid     = String(a["course_id"])
        c_idx   = idxs.course[cid]
        p0      = Int(a["period_start"])  # 0-based

        key = (inst_id, week0, day_str)
        course_map = get!(sessions_by_key_and_course, key,
                          Dict{Int,Vector{Int}}())
        starts = get!(course_map, c_idx, Int[])
        push!(starts, p0)
    end

    metrics = Dict{Tuple{String,Int,String}, Tuple{Int,Int}}()

    for (key, course_map) in sessions_by_key_and_course
        # T = total number of sessions for this instructor/day
        T = 0
        for starts in values(course_map)
            T += length(starts)
        end

        # B = number of b2b edges using (c1 < c2, p2 == p1 + len1) like the MILP
        B = 0
        for (c1, starts1) in course_map
            for (c2, starts2) in course_map
                if c1 >= c2
                    continue
                end
                len1 = parsed.periods_per_session[c1]
                # C8 ensures at most one session per course/day, but we write this
                # generally in case of future changes
                for p1 in starts1
                    for p2 in starts2
                        if p2 == p1 + len1
                            B += 1
                            # Only count one edge per unordered pair per day
                            # (like the model), so break inner loop.
                            break
                        end
                    end
                end
            end
        end

        metrics[key] = (T, B)
    end

    return metrics
end


#############################
# S1: STUDENT CONFLICT TEST #
#############################

"""
Test S1 (student conflicts) via monotonicity:

- Variant A: w1 > 0, w2 = w3 = 0
- Variant B: w1 = w2 = w3 = 0
- Assert conflicts_with_S1 <= conflicts_without_S1
"""
function test_S1_student_conflicts(input_dict::Dict)
    # Variant A: S1 active
    input_S1 = deepcopy(input_dict)
    input_S1["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 10.0,
        "instructor_compactness_weight" => 0.0,
        "preferred_time_slots_weight"   => 0.0,
    )

    # Variant B: all soft constraints off
    input_none = deepcopy(input_dict)
    input_none["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 0.0,
        "instructor_compactness_weight" => 0.0,
        "preferred_time_slots_weight"   => 0.0,
    )

    parsed_S1 = parse_input(input_S1)
    out_S1    = solve_scheduling_problem(input_S1)
    @assert out_S1["status"] in ("optimal", "time_limit")
    assns_S1  = get_assignments(out_S1)
    conflicts_S1 = compute_student_conflicts(assns_S1, parsed_S1)

    parsed_none = parse_input(input_none)
    out_none    = solve_scheduling_problem(input_none)
    @assert out_none["status"] in ("optimal", "time_limit")
    assns_none  = get_assignments(out_none)
    conflicts_none = compute_student_conflicts(assns_none, parsed_none)

    println("S1 test: conflicts with S1=", conflicts_S1,
            ", without S1=", conflicts_none)
    @assert conflicts_S1 <= conflicts_none "S1 violated: student conflicts did not improve when S1 was activated."

    println("✅ S1 student conflict monotonicity test passed")
end

#############################
# S2: B2B PREFERENCE TEST  #
#############################

"""
Test S2 (back-to-back preference) qualitatively:

- Use same dataset, but change w2:
    * Variant A: w2 > 0
    * Variant B: w2 = 0
- For instructors with pref = +1 (dislikes b2b),
  expect B_with_S2 <= B_without_S2 (or at least not larger).
- For instructors with pref = -1 (likes b2b),
  expect B_with_S2 >= B_without_S2 (or at least not smaller).
"""
function test_S2_b2b_preferences(input_dict::Dict)
    input_S2 = deepcopy(input_dict)
    input_S2["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 0.0,
        "instructor_compactness_weight" => 5.0,   # strong
        "preferred_time_slots_weight"   => 0.0,
    )

    input_none = deepcopy(input_dict)
    input_none["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 0.0,
        "instructor_compactness_weight" => 0.0,
        "preferred_time_slots_weight"   => 0.0,
    )

    parsed_S2 = parse_input(input_S2)
    out_S2    = solve_scheduling_problem(input_S2)
    @assert out_S2["status"] in ("optimal", "time_limit")
    assns_S2  = get_assignments(out_S2)
    metrics_S2 = compute_b2b_metrics(assns_S2, parsed_S2)

    parsed_none = parse_input(input_none)
    out_none    = solve_scheduling_problem(input_none)
    @assert out_none["status"] in ("optimal", "time_limit")
    assns_none  = get_assignments(out_none)
    metrics_none = compute_b2b_metrics(assns_none, parsed_none)

    inst_ids = parsed_S2.inst_ids
    ok = true

    for (i_idx, inst_id) in enumerate(inst_ids)
        pref = parsed_S2.inst_b2b_pref[i_idx]
        if pref == 0
            continue
        end
        # Aggregate B over all (week,day) for this instructor
        B_with  = 0
        B_without = 0
        for ((key_inst, week0, day_str), (T,B)) in metrics_S2
            if key_inst == inst_id
                B_with += B
            end
        end
        for ((key_inst, week0, day_str), (T,B)) in metrics_none
            if key_inst == inst_id
                B_without += B
            end
        end

        println("S2: instructor ", inst_id,
                " pref=", pref,
                " B_with_S2=", B_with,
                " B_without_S2=", B_without)

        if pref > 0
            # dislikes b2b: want no more b2b than baseline
            if B_with > B_without
                ok = false
                @warn "S2: instructor $inst_id dislikes b2b but has more b2b edges with S2 than without"
            end
        elseif pref < 0
            # likes b2b: want no fewer b2b than baseline
            if B_with < B_without
                ok = false
                @warn "S2: instructor $inst_id likes b2b but has fewer b2b edges with S2 than without"
            end
        end
    end

    @assert ok "S2 violated: some instructor's b2b behavior did not move in the expected direction."

    println("✅ S2 back-to-back preference qualitative test passed")
end

#############################
# S3: LUNCH PENALTY TEST    #
#############################

"""
Test S3 (lunch-time penalty) via monotonicity:

- Variant A: w3 > 0, w1 = w2 = 0
- Variant B: w3 = 0, w1 = w2 = 0
- Focus only on instructors with inst_lunch_penalty > 0
- Assert lunch_hits_with_S3 <= lunch_hits_without_S3
"""
function test_S3_lunch_penalty(input_dict::Dict)
    input_S3 = deepcopy(input_dict)
    input_S3["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 0.0,
        "instructor_compactness_weight" => 0.0,
        "preferred_time_slots_weight"   => 5.0,
    )

    input_none = deepcopy(input_dict)
    input_none["conflict_weights"] = Dict(
        "global_student_conflict_weight" => 0.0,
        "instructor_compactness_weight" => 0.0,
        "preferred_time_slots_weight"   => 0.0,
    )

    parsed_S3 = parse_input(input_S3)
    out_S3    = solve_scheduling_problem(input_S3)
    @assert out_S3["status"] in ("optimal", "time_limit")
    assns_S3  = get_assignments(out_S3)
    lunch_S3  = compute_lunch_penalty(assns_S3, parsed_S3)

    parsed_none = parse_input(input_none)
    out_none    = solve_scheduling_problem(input_none)
    @assert out_none["status"] in ("optimal", "time_limit")
    assns_none  = get_assignments(out_none)
    lunch_none  = compute_lunch_penalty(assns_none, parsed_none)

    println("S3 test: lunch hits with S3=", lunch_S3,
            ", without S3=", lunch_none)
    @assert lunch_S3 <= lunch_none "S3 violated: lunch-time usage for lunch-averse instructors did not improve when S3 was activated."

    println("✅ S3 lunch penalty monotonicity test passed")
end

#############################
# RUNNERS                   #
#############################

function run_all_hard_constraint_tests(input_dict::Dict)
    println("\n=========== RUNNING HARD CONSTRAINT TESTS (C1–C4, C7–C9) ===========")
    test_C1_teacher_conflict(input_dict)
    test_C2_classroom_conflict(input_dict)
    test_C3_required_sessions_and_weeks(input_dict)
    test_C4_instructor_availability(input_dict)
    test_C7_classroom_capacity(input_dict)
    test_C8_one_session_per_course_per_day(input_dict)
    test_C9_weekly_pattern_consistency(input_dict)
    println("✅ All hard-constraint tests completed\n")
end

function run_all_soft_constraint_tests(input_dict::Dict)
    println("\n=========== RUNNING SOFT CONSTRAINT TESTS (S1, S2, S3) ===========")
    test_S1_student_conflicts(input_dict)
    test_S2_b2b_preferences(input_dict)
    test_S3_lunch_penalty(input_dict)
    println("✅ All soft-constraint tests completed\n")
end

# Example usage (assuming you have `input_json` defined as in your earlier code):
# run_all_hard_constraint_tests(input_json)
# run_all_soft_constraint_tests(input_json)


run_all_soft_constraint_tests (generic function with 2 methods)

### Build Test Datasets

In [67]:
#############################
# DATASET BUILDERS         #
#############################

"""
C1 dataset:
- 1 instructor (I1) teaching 3 courses (C1, C2, C3).
- Availability: full week, all periods.
- Enough rooms & periods so the solver can avoid overlaps.
- Goal: stress teacher conflict C1 (ensure no overlaps for I1).
"""
function build_input_C1()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon", "Wed"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",  # 4 periods of 60 mins
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Room 1", "capacity" => 50),
            Dict("id" => "R2", "name" => "Room 2", "capacity" => 50)
        ],
        "instructors" => [
            Dict(
                "id" => "I1",
                "name" => "C1-only Prof",
                "back_to_back_preference" => 0,
                "allow_lunch_teaching" => true
            )
        ],
        "courses" => [
            Dict("id" => "C1", "name" => "Course 1", "instructor_id" => "I1",
                 "expected_enrollment" => 30, "type" => "full_term"),
            Dict("id" => "C2", "name" => "Course 2", "instructor_id" => "I1",
                 "expected_enrollment" => 30, "type" => "full_term"),
            Dict("id" => "C3", "name" => "Course 3", "instructor_id" => "I1",
                 "expected_enrollment" => 30, "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["C1", "C2"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["C2", "C3"]),
            Dict("id" => "S3", "enrolled_course_ids" => ["C1", "C3"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C2 dataset:
- 2 rooms, 4 courses at the same enrollment.
- 2 instructors with full availability.
- The solver should spread simultaneous sessions across rooms, not stack them in one.
- Stresses classroom conflict C2.
"""
function build_input_C2()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 1,
            "days" => ["Mon"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Room 1", "capacity" => 40),
            Dict("id" => "R2", "name" => "Room 2", "capacity" => 40)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof A",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "Prof B",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "C1", "name" => "C1", "instructor_id" => "I1",
                 "expected_enrollment" => 30, "type" => "full_term"),
            Dict("id" => "C2", "name" => "C2", "instructor_id" => "I1",
                 "expected_enrollment" => 30, "type" => "full_term"),
            Dict("id" => "C3", "name" => "C3", "instructor_id" => "I2",
                 "expected_enrollment" => 30, "type" => "full_term"),
            Dict("id" => "C4", "name" => "C4", "instructor_id" => "I2",
                 "expected_enrollment" => 30, "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["C1", "C3"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["C2", "C4"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C3 dataset:
- Mix of full_term, first_half_term, second_half_term.
- Ensures correct total_sessions and week ranges.
"""
function build_input_C3()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 6,
            "days" => ["Mon", "Wed"],
            "day_start_time" => "09:00",
            "day_end_time" => "15:00",
            "period_length_minutes" => 90
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Main", "capacity" => 100)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof Full",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "Prof FirstHalf",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I3", "name" => "Prof SecondHalf",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "FULL101",  "name" => "Full Term",
                 "instructor_id" => "I1", "expected_enrollment" => 60,
                 "type" => "full_term"),
            Dict("id" => "FH101",    "name" => "First Half",
                 "instructor_id" => "I2", "expected_enrollment" => 40,
                 "type" => "first_half_term"),
            Dict("id" => "SH101",    "name" => "Second Half",
                 "instructor_id" => "I3", "expected_enrollment" => 40,
                 "type" => "second_half_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["FULL101", "FH101"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["FULL101", "SH101"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C4 dataset:
- Instructor I1 only available Mon morning (period 0 & 1).
- Course C1 must be placed within those slots.
- Stresses the instructor availability mask.
"""
function build_input_C4()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon", "Tue"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Room 1", "capacity" => 40)
        ],
        "instructors" => [
            Dict(
                "id" => "I1",
                "name" => "Morning-Only Prof",
                "back_to_back_preference" => 0,
                "allow_lunch_teaching" => true,
                "availability" => [
                    Dict("day" => "Mon", "period_index" => 0),
                    Dict("day" => "Mon", "period_index" => 1)
                ]
            )
        ],
        "courses" => [
            Dict("id" => "C1", "name" => "Morning Course",
                 "instructor_id" => "I1", "expected_enrollment" => 30,
                 "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["C1"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C7 dataset:
- One small room (cap=20), one big (cap=100).
- One large course (enr=80) and one small course (enr=10).
- Tests that large course never goes into the small room.
"""
function build_input_C7()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 1,
            "days" => ["Mon"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "SmallR", "name" => "Small Room", "capacity" => 20),
            Dict("id" => "BigR", "name" => "Big Room", "capacity" => 100)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof Big",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "Prof Small",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "BIG101", "name" => "Big Class",
                 "instructor_id" => "I1", "expected_enrollment" => 80,
                 "type" => "full_term"),
            Dict("id" => "SM101", "name" => "Small Class",
                 "instructor_id" => "I2", "expected_enrollment" => 10,
                 "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["BIG101"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["SM101"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C8 dataset:
- Course C1: full_term needing 2 sessions/week (via term_config + calculate_sessions).
  (Here we keep same structure, but the key feature is multiple courses and days.)
- Enough days so C1 must use at most one session/day.
- Small but exercises C8 check.
"""
function build_input_C8()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon", "Tue", "Wed"],
            "day_start_time" => "09:00",
            "day_end_time" => "15:00",
            "period_length_minutes" => 90
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Room 1", "capacity" => 50)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "C1", "name" => "C1",
                 "instructor_id" => "I1",
                 "expected_enrollment" => 40,
                 "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["C1"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
C9 dataset:
- Course C1 active weeks 1–4 (full_term), course C2 first_half_term.
- Enough structure so the pattern consistency across weeks matters.
"""
function build_input_C9()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 4,
            "days" => ["Mon", "Wed"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Main", "capacity" => 80)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof Full",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "Prof Half",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "FULL201", "name" => "Full Term",
                 "instructor_id" => "I1",
                 "expected_enrollment" => 50,
                 "type" => "full_term"),
            Dict("id" => "FH201", "name" => "First Half",
                 "instructor_id" => "I2",
                 "expected_enrollment" => 30,
                 "type" => "first_half_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["FULL201", "FH201"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

# ---------- Soft constraints datasets ----------

"""
S1 dataset:
- 3 heavily overlapping courses C1,C2,C3 with many shared students.
- Enough days/rooms to move them apart.
- Designed so S1>0 should separate them more than S1=0.
"""
function build_input_S1()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon", "Tue"],
            "day_start_time" => "09:00",
            "day_end_time" => "13:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "R1", "capacity" => 80),
            Dict("id" => "R2", "name" => "R2", "capacity" => 80)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "Prof1",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "Prof2",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I3", "name" => "Prof3",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "C1", "name" => "C1", "instructor_id" => "I1",
                 "expected_enrollment" => 50, "type" => "full_term"),
            Dict("id" => "C2", "name" => "C2", "instructor_id" => "I2",
                 "expected_enrollment" => 50, "type" => "full_term"),
            Dict("id" => "C3", "name" => "C3", "instructor_id" => "I3",
                 "expected_enrollment" => 50, "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["C1", "C2", "C3"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["C1", "C2"]),
            Dict("id" => "S3", "enrolled_course_ids" => ["C2", "C3"]),
            Dict("id" => "S4", "enrolled_course_ids" => ["C1", "C3"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 1.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
S2 dataset:
- Instructor I1 (pref=-1, likes b2b) teaches A,B,C.
- Instructor I2 (pref=+1, dislikes b2b) teaches D,E.
- Availability structured so:
    - I1 can create b2b chains.
    - I2 has options to avoid b2b.
"""
function build_input_S2()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon"],
            "day_start_time" => "09:00",
            "day_end_time" => "15:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "R1", "capacity" => 80),
            Dict("id" => "R2", "name" => "R2", "capacity" => 80)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "LikesB2B",
                 "back_to_back_preference" => -1,
                 "allow_lunch_teaching" => true),
            Dict("id" => "I2", "name" => "HatesB2B",
                 "back_to_back_preference" => 1,
                 "allow_lunch_teaching" => true)
        ],
        "courses" => [
            Dict("id" => "A", "name" => "A", "instructor_id" => "I1",
                 "expected_enrollment" => 20, "type" => "full_term"),
            Dict("id" => "B", "name" => "B", "instructor_id" => "I1",
                 "expected_enrollment" => 20, "type" => "full_term"),
            Dict("id" => "C", "name" => "C", "instructor_id" => "I1",
                 "expected_enrollment" => 20, "type" => "full_term"),
            Dict("id" => "D", "name" => "D", "instructor_id" => "I2",
                 "expected_enrollment" => 20, "type" => "full_term"),
            Dict("id" => "E", "name" => "E", "instructor_id" => "I2",
                 "expected_enrollment" => 20, "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["A", "B"]),
            Dict("id" => "S2", "enrolled_course_ids" => ["B", "C"]),
            Dict("id" => "S3", "enrolled_course_ids" => ["D", "E"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 5.0,
            "preferred_time_slots_weight" => 0.0
        )
    )
end

"""
S3 dataset:
- I1 dislikes lunch teaching, I2 is neutral.
- Courses for I1/I2 have flexibility across lunch and non-lunch periods.
- Designed so w3>0 pushes I1 away from lunch, while I2 is indifferent.
"""
function build_input_S3()
    return Dict(
        "term_config" => Dict(
            "num_weeks" => 2,
            "days" => ["Mon"],
            "day_start_time" => "09:00",
            "day_end_time" => "17:00",
            "period_length_minutes" => 60
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "R1", "capacity" => 80)
        ],
        "instructors" => [
            Dict("id" => "I1", "name" => "NoLunchProf",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => false),   # penalty>0
            Dict("id" => "I2", "name" => "LunchOKProf",
                 "back_to_back_preference" => 0,
                 "allow_lunch_teaching" => true)     # penalty=0
        ],
        "courses" => [
            Dict("id" => "LUNCH_SENSITIVE", "name" => "L_S",
                 "instructor_id" => "I1",
                 "expected_enrollment" => 30,
                 "type" => "full_term"),
            Dict("id" => "NEUTRAL", "name" => "Neutral",
                 "instructor_id" => "I2",
                 "expected_enrollment" => 30,
                 "type" => "full_term")
        ],
        "students" => [
            Dict("id" => "S1", "enrolled_course_ids" => ["LUNCH_SENSITIVE", "NEUTRAL"])
        ],
        "conflict_weights" => Dict(
            "global_student_conflict_weight" => 0.0,
            "instructor_compactness_weight" => 0.0,
            "preferred_time_slots_weight" => 5.0
        )
    )
end

#############################
# MINIMAL RUNNERS          #
#############################

# tiny helper to avoid repeating assignment extraction
get_assignments(output::Dict) = output["schedule"]["assignments"]::Vector{Any}

function run_C1_dataset()
    input = build_input_C1()
    println("\n=== C1 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C1_teacher_conflict(input)
end

function run_C2_dataset()
    input = build_input_C2()
    println("\n=== C2 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C2_classroom_conflict(input)
end

function run_C3_dataset()
    input = build_input_C3()
    println("\n=== C3 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C3_required_sessions_and_weeks(input)
end

function run_C4_dataset()
    input = build_input_C4()
    println("\n=== C4 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C4_instructor_availability(input)
end

function run_C7_dataset()
    input = build_input_C7()
    println("\n=== C7 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C7_classroom_capacity(input)
end

function run_C8_dataset()
    input = build_input_C8()
    println("\n=== C8 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C8_one_session_per_course_per_day(input)
end

function run_C9_dataset()
    input = build_input_C9()
    println("\n=== C9 DATASET ===")
    out = solve_scheduling_problem(input)
    println("status = ", out["status"])
    test_C9_weekly_pattern_consistency(input)
end

function run_S1_dataset()
    input = build_input_S1()
    println("\n=== S1 DATASET ===")
    run_all_soft_constraint_tests(input)  # uses S1,S2,S3 tests; or call test_S1_student_conflicts(input)
end

function run_S2_dataset()
    input = build_input_S2()
    println("\n=== S2 DATASET ===")
    run_all_soft_constraint_tests(input)  # or just test_S2_b2b_preferences(input)
end

function run_S3_dataset()
    input = build_input_S3()
    println("\n=== S3 DATASET ===")
    run_all_soft_constraint_tests(input)  # or just test_S3_lunch_penalty(input)
end

"""
Optional: run everything once.
Comment out any you don't care about right now.
"""
function run_all_constraint_datasets()
    run_C1_dataset()
    run_C2_dataset()
    run_C3_dataset()
    run_C4_dataset()
    run_C7_dataset()
    run_C8_dataset()
    run_C9_dataset()

    run_S1_dataset()
    run_S2_dataset()
    run_S3_dataset()
end


run_all_constraint_datasets

In [68]:
run_S2_dataset()


=== S2 DATASET ===

=========== RUNNING SOFT CONSTRAINT TESTS (S1, S2, S3) ===========
📊 Julia solver started at 2025-12-03T18:29:36.287
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 5 courses, 2 weeks, 1 days, 6 periods
  Constraints: 461
  Variables:   1048
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.003s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = 0.0
  S3_lunch_penalty     = 0.0
  Total objective      = 0.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
Instructor I1 pref = -1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Mon: T=3.0, edges(E)=T-1=2.0, B=1.0, contrib=-0.0
  Week 2, Day Mon: T=3.0, edges(E)=T-1=2.0, B=1.0, contrib=-0.0
Instructor I2 pref = 1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Mon: T=2.0, edges

In [69]:
run_all_constraint_datasets()



=== C1 DATASET ===
📊 Julia solver started at 2025-12-03T18:29:41.591
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 3 courses, 2 weeks, 2 days, 4 periods
  Constraints: 407
  Variables:   404
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.002s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = 0.0
  S3_lunch_penalty     = 0.0
  Total objective      = 0.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
=== Assignments ===
Course C1 in room R2 | week=1 day=Wed period_start=3 (len=2)
Course C1 in room R2 | week=2 day=Wed period_start=3 (len=2)
Course C2 in room R2 | week=1 day=Wed period_start=1 (len=2)
Course C2 in room R2 | week=2 day=Wed period_start=1 (len=2)
Course C3 in room R2 | week=1 day=Mon period_start=3 (len=2)
Course C3 in r

# What-If Analysis

## What-If Functions

In [12]:
# =================================================================================
# WHAT-IF ANALYSIS / COUNTERFACTUAL REASONING (X-MILP / UDSP)
# =================================================================================

"""
Solve a What-If / counterfactual query using X-MILP's UDSP construction.

UDSP(M, CQ) = CSP with:
  (i)   f(x) ≤ f*            (minimality vs original solution)
  (ii)  C                    (all original hard constraints)
  (iii) CQ                   (query constraints from the user)

- If UDSP is infeasible: compute IIS to explain which query constraints (and/or
  minimality) are incompatible.
- If UDSP is feasible: return the alternative schedule and its objective value.
"""
function solve_what_if_query(
    input_dict::Dict,
    query_constraints::Vector,      # Vector{Dict} with query specs
    original_objective::Float64     # f* from the original optimal solution
)
    try
        println("🔍 Solving What-If Query (UDSP)...")
        setup_gurobi_license()

        # Re-parse input and rebuild a "fresh" model
        parsed = parse_input(input_dict)
        C = length(parsed.course_ids)
        I = length(parsed.inst_ids)
        R = length(parsed.room_ids)
        W = parsed.num_weeks
        D = length(parsed.days)
        P = parsed.num_periods

        model = Model(Gurobi.Optimizer)
        set_optimizer_attribute(model, "OutputFlag", 0)
        set_optimizer_attribute(model, "TimeLimit", 300)

        # --- Decision variables (same structure as original model) ---
        @variable(model, x[1:C, 1:W, 1:D, 1:P, 1:R], Bin)       # course starts
        @variable(model, h[1:I, 1:W, 1:D, 1:P], Bin)            # instructor teaching per period
        @variable(model, φ[c1 in 1:C, c2 in (c1+1):C, 1:W, 1:D, 1:P], Bin)  # S1 conflicts
        @variable(model, π[1:C, 1:W, 1:D, 1:P], Bin)            # S3 lunch
        @variable(model, z[1:I, 1:C, 1:C, 1:W, 1:D, 1:P], Bin)  # S2 b2b "edges"
        @variable(model, b2b_sess[1:I, 1:C, 1:C, 1:W, 1:D], Bin) # S2 b2b per pair/day
        @variable(model, teach_c[1:I, 1:C, 1:W, 1:D], Bin)      # course taught that day
        @variable(model, has_teaching[1:I, 1:W, 1:D], Bin)      # instructor teaches at least one course that day

        println("  Adding original hard constraints (C)...")
        add_hard_constraints!(model, x, h, parsed, C, I, R, W, D, P)

        println("  Adding soft constraint structure and objective...")
        obj_s1, obj_s2, obj_s3 = build_soft_objective(
            model, x, h, b2b_sess, z, π, φ, teach_c, has_teaching,
            parsed, C, I, R, W, D, P
        )

        println("  Adding query constraints (CQ)...")
        query_constraint_refs = add_query_constraints!(
            model, query_constraints, x, parsed, C, R, W, D, P
        )

        println("  Adding minimality constraint: f(x) ≤ $original_objective")
        @constraint(model, minimality, obj_s1 + obj_s2 + obj_s3 <= original_objective)

        # UDSP objective is the same f(x) as the original problem
        @objective(model, Min, obj_s1 + obj_s2 + obj_s3)

        println("  🚀 Optimizing UDSP...")
        optimize!(model)

        status = termination_status(model)
        solve_time_val = solve_time(model)

        println("  Status:     $status")
        println("  Solve time: $(round(solve_time_val, digits=2))s")

        if status == MOI.INFEASIBLE || status == MOI.INFEASIBLE_OR_UNBOUNDED
            # Query scenario is infeasible w.r.t. constraints + minimality
            println("  ❌ UDSP is infeasible - computing IIS...")
            iis_result = compute_iis_explanation(model, query_constraint_refs, minimality)

            return Dict(
                "status" => "infeasible_query",
                "query_feasible" => false,
                "solve_time_seconds" => solve_time_val,
                "iis" => iis_result["iis_constraints"],
                "iis_summary" => iis_result["summary"],
                "explanation" =>
                    "The what-if scenario cannot achieve an objective ≤ original optimum. " *
                    "See IIS for conflicting constraints.",
                "original_objective" => original_objective,
                "metadata" => Dict(
                    "solver" => "Gurobi (Julia/JuMP)",
                    "method" => "UDSP (X-MILP)",
                    "timestamp" => string(now())
                )
            )

        elseif status == MOI.OPTIMAL || status == MOI.TIME_LIMIT
            println("  ✅ UDSP has a feasible solution")

            # Reuse your existing formatter to get schedule etc.
            alternative_output = format_output(model, x, parsed)
            new_objective = has_values(model) ? objective_value(model) : nothing
            Δ = (new_objective === nothing) ? nothing : (new_objective - original_objective)

            return Dict(
                "status" => "feasible_query",
                "query_feasible" => true,
                "solve_time_seconds" => solve_time_val,
                "original_objective" => original_objective,
                "alternative_objective" => new_objective,
                "objective_difference" => Δ,
                "alternative_schedule" => alternative_output["schedule"],
                "alternative_soft_constraints" => alternative_output["soft_constraint_summary"],
                "explanation" =>
                    "The what-if scenario is feasible. Objective difference: " *
                    (Δ === nothing ? "N/A" : string(round(Δ, digits=2))),
                "metadata" => Dict(
                    "solver" => "Gurobi (Julia/JuMP)",
                    "method" => "UDSP (X-MILP)",
                    "timestamp" => string(now())
                )
            )
        else
            return Dict(
                "status" => "udsp_error",
                "query_feasible" => false,
                "solve_time_seconds" => solve_time_val,
                "explanation" => "UDSP solver terminated with status: $(string(status))",
                "metadata" => Dict(
                    "solver" => "Gurobi (Julia/JuMP)",
                    "solver_status" => string(status)
                )
            )
        end

    catch e
        error_msg = string(e)
        println("❌ What-if query error: $error_msg")

        traceback_str = try
            sprint(showerror, e, catch_backtrace())
        catch
            "Stack trace unavailable"
        end

        return Dict(
            "status" => "error",
            "query_feasible" => false,
            "solve_time_seconds" => 0.0,
            "explanation" => "Error during what-if analysis: $error_msg",
            "diagnostics" => Dict(
                "error" => error_msg,
                "traceback" => traceback_str
            ),
            "metadata" => Dict(
                "solver" => "Gurobi (Julia/JuMP)",
                "timestamp" => string(now())
            )
        )
    end
end


# -----------------------------------------------------------------------------
# Query → constraint encoder (CQ)
# -----------------------------------------------------------------------------

"""
Add query constraints to the UDSP model.

Each element of `query_constraints` is a Dict like:

  Dict("type" => "enforce_time_slot",
       "course_id" => "CME307",
       "week" => 0,          # 0-indexed
       "day" => "Mon",
       "period_start" => 2)  # 0-indexed

Returns a list of (idx, qtype, constraint_ref) tuples to check IIS membership.
"""
function add_query_constraints!(
    model,
    query_constraints,
    x,
    parsed,
    C, R, W, D, P
)
    constraint_refs = Any[]

    for (idx, qc) in enumerate(query_constraints)
        qtype        = qc["type"]
        course_id    = get(qc, "course_id", nothing)
        week         = get(qc, "week", nothing)
        day          = get(qc, "day", nothing)
        period_start = get(qc, "period_start", nothing)
        period_end   = get(qc, "period_end", nothing)
        room_id      = get(qc, "room_id", nothing)

        # Map IDs to indices
        c_idx = course_id === nothing ? nothing : findfirst(==(course_id), parsed.course_ids)
        d_idx = day       === nothing ? nothing : findfirst(==(day),       parsed.days)
        r_idx = room_id   === nothing ? nothing : findfirst(==(room_id),   parsed.room_ids)
        w_idx = week      === nothing ? nothing : (week + 1)       # external is 0-based
        p_idx = period_start === nothing ? nothing : (period_start + 1)

        if qtype == "enforce_time_slot"
            # Course must be at (w,d,p) in some room
            if c_idx !== nothing && w_idx !== nothing && d_idx !== nothing && p_idx !== nothing
                ref = @constraint(model, sum(x[c_idx, w_idx, d_idx, p_idx, r] for r in 1:R) == 1)
                push!(constraint_refs, (idx, "enforce_time_slot", ref))
                println("    Added: Enforce $course_id at week=$week day=$day period=$period_start")
            end

        elseif qtype == "veto_time_slot"
            # Course must NOT be at (w,d,p)
            if c_idx !== nothing && d_idx !== nothing && p_idx !== nothing
                if w_idx !== nothing
                    ref = @constraint(model,
                        sum(x[c_idx, w_idx, d_idx, p_idx, r] for r in 1:R) == 0)
                    push!(constraint_refs, (idx, "veto_time_slot", ref))
                else
                    # veto across all weeks
                    ref = @constraint(model,
                        sum(x[c_idx, w, d_idx, p_idx, r] for w in 1:W, r in 1:R) == 0)
                    push!(constraint_refs, (idx, "veto_time_slot_all_weeks", ref))
                end
                println("    Added: Veto $course_id on day=$day period=$period_start")
            end

        elseif qtype == "veto_day"
            # Course cannot be on this day at all
            if c_idx !== nothing && d_idx !== nothing
                ref = @constraint(model,
                    sum(x[c_idx, w, d_idx, p, r] for w in 1:W, p in 1:P, r in 1:R) == 0)
                push!(constraint_refs, (idx, "veto_day", ref))
                println("    Added: Veto $course_id on $day")
            end

        elseif qtype == "enforce_room"
            # Course must use this room at least once
            if c_idx !== nothing && r_idx !== nothing
                ref = @constraint(model,
                    sum(x[c_idx, w, d, p, r_idx] for w in 1:W, d in 1:D, p in 1:P) >= 1)
                push!(constraint_refs, (idx, "enforce_room", ref))
                println("    Added: Enforce $course_id in room $room_id")
            end

        elseif qtype == "enforce_before_time"
            # All sessions of course must finish before period_end
            if c_idx !== nothing && period_end !== nothing
                p_end_idx = period_end + 1
                dur       = parsed.periods_per_session[c_idx]
                # Only start times that finish by p_end_idx
                ref = @constraint(model,
                    sum(
                        x[c_idx, w, d, p, r]
                        for w in 1:W, d in 1:D, p in 1:p_end_idx, r in 1:R
                        if p + dur - 1 <= p_end_idx
                    ) >= parsed.total_sessions[c_idx]
                )
                push!(constraint_refs, (idx, "enforce_before_time", ref))
                println("    Added: Enforce $course_id before period $period_end")
            end

        elseif qtype == "enforce_after_time"
            # All sessions of course must start at/after period_start
            if c_idx !== nothing && period_start !== nothing
                p_start_idx = period_start + 1
                dur         = parsed.periods_per_session[c_idx]
                ref = @constraint(model,
                    sum(
                        x[c_idx, w, d, p, r]
                        for w in 1:W, d in 1:D, p in p_start_idx:P, r in 1:R
                        if p + dur - 1 <= P
                    ) >= parsed.total_sessions[c_idx]
                )
                push!(constraint_refs, (idx, "enforce_after_time", ref))
                println("    Added: Enforce $course_id after period $period_start")
            end
        end
    end

    println("  Added $(length(constraint_refs)) query constraints")
    return constraint_refs
end


# -----------------------------------------------------------------------------
# IIS extraction (X-MILP step 2)
# -----------------------------------------------------------------------------

"""
Compute IIS and map which constraints are "in conflict".

We specifically check:
  - whether the minimality constraint is in the IIS
  - which query constraints are in the IIS
"""
function compute_iis_explanation(model, query_constraint_refs, minimality_constraint)
    println("  Computing IIS (Irreducible Infeasible Subsystem)...")

    try
        compute_conflict!(model)
        println("  ✅ IIS computation successful")

        iis_constraints = Any[]

        # Minimality constraint in IIS?
        if MOI.get(model, MOI.ConstraintConflictStatus(), minimality_constraint) == MOI.IN_CONFLICT
            push!(iis_constraints, Dict(
                "id" => "minimality",
                "type" => "minimality",
                "description" => "Objective must be ≤ original optimal value",
                "in_iis" => true,
                "reason" => "Cannot achieve ≤ original objective together with query constraints"
            ))
        end

        # Query constraints in IIS?
        for (idx, qtype, ref) in query_constraint_refs
            in_conflict = MOI.get(model, MOI.ConstraintConflictStatus(), ref) == MOI.IN_CONFLICT
            if in_conflict
                push!(iis_constraints, Dict(
                    "id" => "query_$idx",
                    "type" => qtype,
                    "description" => "User query constraint #$idx of type $qtype",
                    "in_iis" => true
                ))
            end
        end

        num_conflicts  = length(iis_constraints)
        query_conflicts = count(c -> startswith(get(c, "id", ""), "query"), iis_constraints)
        has_minimality = any(c -> get(c, "id", "") == "minimality", iis_constraints)

        summary = Dict(
            "num_constraints_in_iis" => num_conflicts,
            "num_query_constraints_in_iis" => query_conflicts,
            "minimality_in_iis" => has_minimality,
            "interpretation" => has_minimality ?
                "Scenario conflicts with achieving the original objective value." :
                "Scenario conflicts with hard constraints only."
        )

        return Dict(
            "iis_constraints" => iis_constraints,
            "summary" => summary
        )

    catch e
        println("  ⚠️ IIS computation failed: $(string(e))")
        return Dict(
            "iis_constraints" => [],
            "summary" => Dict(
                "num_constraints_in_iis" => 0,
                "error" => "IIS computation failed: $(string(e))"
            )
        )
    end
end


# -----------------------------------------------------------------------------
# Shared hard-constraint builder (C1–C4, C7, C8, C9)
# -----------------------------------------------------------------------------

"""
Add all hard constraints C1–C4, C7, C8, C9 to `model` using decision vars (x, h).
Factored out so we can reuse it both in the main solve and in UDSP.
"""
function add_hard_constraints!(
    model,
    x,
    h,
    parsed,
    C, I, R, W, D, P
)
    # Helper: valid start periods for a given "current" period
    function valid_starts(c, current_p)
        dur = parsed.periods_per_session[c]
        return max(1, current_p - dur + 1):min(current_p, P)
    end

    # C1: Teacher conflict + link to h
    for i in 1:I, w in 1:W, d in 1:D, p in 1:P
        my_courses = [c for c in 1:C if parsed.course_inst[c] == i]
        occ = @expression(model,
            sum(x[c,w,d,s,r]
                for c in my_courses, r in 1:R, s in valid_starts(c, p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= 1)
        @constraint(model, h[i,w,d,p] == occ)
    end

    # C2: Classroom conflict
    for r in 1:R, w in 1:W, d in 1:D, p in 1:P
        @constraint(model,
            sum(x[c,w,d,s,r]
                for c in 1:C, s in valid_starts(c, p)
                if s + parsed.periods_per_session[c] - 1 <= P) <= 1
        )
    end

    # C3: class hours, duration & active weeks
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        dur     = parsed.periods_per_session[c]
        req_s   = parsed.total_sessions[c]

        @constraint(model,
            sum(x[c,w,d,p,r]
                for w in w_start:w_end, d in 1:D, p in 1:P, r in 1:R
                if p + dur - 1 <= P) == req_s
        )

        all_weeks      = 1:W
        inactive_weeks = setdiff(all_weeks, w_start:w_end)
        for w in inactive_weeks
            @constraint(model,
                sum(x[c,w,d,p,r] for d in 1:D, p in 1:P, r in 1:R) == 0
            )
        end
    end

    # C4: instructor availability
    for c in 1:C
        inst = parsed.course_inst[c]
        dur  = parsed.periods_per_session[c]
        for w in 1:W, d in 1:D, p in 1:P, r in 1:R
            if p + dur - 1 <= P
                is_avail = all(parsed.avail[inst,d,t] for t in p:(p+dur-1))
                if !is_avail
                    @constraint(model, x[c,w,d,p,r] == 0)
                end
            else
                @constraint(model, x[c,w,d,p,r] == 0)
            end
        end
    end

    # C7: classroom capacity
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        @constraint(model, parsed.course_enr[c] * x[c,w,d,p,r] <= parsed.room_cap[r])
    end

    # C8: at most one session per course/day
    for c in 1:C, w in 1:W, d in 1:D
        dur = parsed.periods_per_session[c]
        @constraint(model,
            sum(x[c,w,d,p,r] for p in 1:P, r in 1:R if p + dur - 1 <= P) <= 1
        )
    end

    # C9: weekly pattern consistency
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        if w_end > w_start
            for w in (w_start+1):w_end, d in 1:D, p in 1:P, r in 1:R
                @constraint(model, x[c,w,d,p,r] == x[c,w_start,d,p,r])
            end
        end
    end
end


# -----------------------------------------------------------------------------
# Shared soft-constraint objective builder (S1, S2, S3)
# -----------------------------------------------------------------------------

"""
Build S1 (student conflict), S2 (b2b), S3 (lunch) objective terms and all
linking constraints for φ, z, b2b_sess, teach_c, has_teaching, π.

Return (obj_s1, obj_s2, obj_s3) such that:

    f(x) = obj_s1 + obj_s2 + obj_s3

is identical to the objective in `build_and_solve_model`.
"""
function build_soft_objective(
    model,
    x,
    h,
    b2b_sess,
    z,
    π,
    φ,
    teach_c,
    has_teaching,
    parsed,
    C, I, R, W, D, P
)
    # Helper
    function valid_starts(c, current_p)
        dur = parsed.periods_per_session[c]
        return max(1, current_p - dur + 1):min(current_p, P)
    end

    # --- S1: student conflicts ---
    obj_s1 = @expression(model,
        sum(parsed.w1 * parsed.students_cc[c1,c2] * φ[c1,c2,w,d,p]
            for c1 in 1:C, c2 in (c1+1):C, w in 1:W, d in 1:D, p in 1:P
            if parsed.students_cc[c1,c2] > 0)
    )

    # conflict detection constraints for φ
    for c1 in 1:C, c2 in (c1+1):C
        if parsed.students_cc[c1,c2] > 0
            for w in 1:W, d in 1:D, p in 1:P
                occ1 = @expression(model,
                    sum(x[c1,w,d,s,r]
                        for r in 1:R, s in valid_starts(c1,p)
                        if s + parsed.periods_per_session[c1] - 1 <= P)
                )
                occ2 = @expression(model,
                    sum(x[c2,w,d,s,r]
                        for r in 1:R, s in valid_starts(c2,p)
                        if s + parsed.periods_per_session[c2] - 1 <= P)
                )
                @constraint(model, occ1 + occ2 <= 1 + φ[c1,c2,w,d,p])
            end
        end
    end

    # --- S2: back-to-back (symmetric) ---
    # link z and b2b_sess
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c1,w,d,p,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c2,w,d,p+len1,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] >=
                            sum(x[c1,w,d,p,r]      for r in 1:R) +
                            sum(x[c2,w,d,p+len1,r] for r in 1:R) - 1
                    )
                end
            end
        end
    end

    # b2b_sess is OR of z over p
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                @constraint(model,
                    b2b_sess[i,c1,c2,w,d] <=
                        sum(z[i,c1,c2,w,d,p] for p in 1:(P-len1))
                )
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= b2b_sess[i,c1,c2,w,d]
                    )
                end
            end
        end
    end

    # teach_c: whether instructor teaches course c that day
    for i in 1:I, c in 1:C, w in 1:W, d in 1:D
        if parsed.course_inst[c] != i
            @constraint(model, teach_c[i,c,w,d] == 0)
        else
            total = @expression(model,
                sum(x[c,w,d,p,r] for p in 1:P, r in 1:R)
            )
            @constraint(model, teach_c[i,c,w,d] <= total)
            @constraint(model, teach_c[i,c,w,d] >= total / P)
        end
    end

    # has_teaching: ≥1 course that day?
    for i in 1:I, w in 1:W, d in 1:D
        T_sum = @expression(model,
            sum(teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
        )
        max_T = sum(1 for c in 1:C if parsed.course_inst[c] == i)
        if max_T > 0
            @constraint(model, has_teaching[i,w,d] <= T_sum)
            @constraint(model, has_teaching[i,w,d] >= T_sum / max_T)
        else
            @constraint(model, has_teaching[i,w,d] == 0)
        end
    end

    # S2 objective: w2 * pref_i * (2B - (T-1)) * has_teaching
    obj_s2_terms = Any[]
    for i in 1:I, w in 1:W, d in 1:D
        pref = parsed.inst_b2b_pref[i]
        if pref == 0
            continue
        end
        T_expr = @expression(model,
            sum(teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
        )
        B_expr = @expression(model,
            sum(b2b_sess[i,c1,c2,w,d]
                for c1 in 1:(C-1), c2 in (c1+1):C
                if parsed.course_inst[c1] == i && parsed.course_inst[c2] == i)
        )
        push!(obj_s2_terms,
            parsed.w2 * pref * has_teaching[i,w,d] * (2*B_expr - (T_expr - 1))
        )
    end
    obj_s2 = @expression(model, sum(obj_s2_terms))

    # --- S3: lunch penalties ---
    lunch_periods = get_lunch_periods(parsed.term_config, P)
    obj_s3 = @expression(model,
        sum(
            parsed.w3 *
            parsed.inst_lunch_penalty[parsed.course_inst[c]] *
            π[c,w,d,p]
            for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods
        )
    )

    # Link π to x
    for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods
        occ = @expression(model,
            sum(x[c,w,d,s,r]
                for r in 1:R, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= π[c,w,d,p])
    end

    return obj_s1, obj_s2, obj_s3
end


build_soft_objective

## Test What-If Analysis

In [70]:
# =================================================================================
# WHAT-IF TEST HARNESS
# =================================================================================

"""
Pick a "representative" assignment from the baseline schedule to use in tests.

We take:
  - first assignment overall
  - first course that has >1 session (if any) to test day/room stuff.
"""
function pick_test_assignments(schedule::Vector{Any})
    if isempty(schedule)
        error("Baseline schedule is empty – cannot build what-if tests.")
    end

    first = schedule[1]

    # group by course_id to find multi-session course
    by_course = Dict{String,Vector{Dict}}()
    for a_any in schedule
        a = a_any::Dict{String,Any}
        cid = a["course_id"]
        if !haskey(by_course, cid)
            by_course[cid] = Dict{String,Any}[]
        end
        push!(by_course[cid], a)
    end

    multi_course = nothing
    for (cid, assns) in by_course
        if length(assns) > 1
            multi_course = (cid, assns)
            break
        end
    end

    return first, multi_course
end

"""
Pretty-print a compact summary of a what-if result.
"""
function print_what_if_summary(label::String, res::Dict)
    println("------------------------------------------------------------")
    println("CASE: $label")
    println("  status           = ", res["status"])
    println("  query_feasible   = ", get(res, "query_feasible", missing))
    println("  solve_time (s)   = ", get(res, "solve_time_seconds", missing))
    println("  explanation      = ", get(res, "explanation", ""))
    if haskey(res, "alternative_objective")
        println("  f*               = ", get(res, "original_objective", missing))
        println("  f_alt            = ", get(res, "alternative_objective", missing))
        println("  Δf = f_alt - f*  = ", get(res, "objective_difference", missing))
    end
    if get(res, "status", "") == "infeasible_query" && haskey(res, "iis_summary")
        println("  IIS summary      = ", res["iis_summary"])
        if haskey(res, "iis")
            println("  IIS constraints  = ", res["iis"])
        end
    end
end

"""
Find a (course, week0, day_str, period_start0) combination such that the
instructor of that course is *completely unavailable* on that day
(i.e., parsed.avail[i, d, p] == false for all p).

Returns `nothing` if no such combination exists.
"""
function find_course_on_completely_unavailable_day(parsed)
    C = length(parsed.course_ids)
    W = parsed.num_weeks
    D = length(parsed.days)
    P = parsed.num_periods

    for c in 1:C
        inst = parsed.course_inst[c]
        for d in 1:D
            # Instructor is never available on this day?
            if all(!parsed.avail[inst, d, p] for p in 1:P)
                # Use the first active week for this course
                w_start = parsed.week_starts[c]      # 1-based
                week0   = w_start - 1                # external is 0-based
                day_str = String(parsed.days[d])
                period_start0 = 0                    # 0-based; any period will be infeasible

                return (
                    parsed.course_ids[c],  # course_id
                    week0,                 # 0-based
                    day_str,               # day name
                    period_start0          # 0-based
                )
            end
        end
    end

    return nothing
end



find_course_on_completely_unavailable_day

In [71]:
"""
Run a suite of "what-if" test cases for all query types.

This is a *smoke test* / exploration tool, not a strict unit test:
it runs each query, prints its status and IIS if infeasible.
"""
function run_what_if_tests(input_dict::Dict)
    println("\n================ BASELINE SOLVE ================")
    base_output = solve_scheduling_problem(input_dict)

    if base_output["status"] != "optimal" && base_output["status"] != "time_limit"
        println("Baseline solve not successful: ", base_output["status"])
        return
    end

    f_star = base_output["objective_value"]
    schedule = base_output["schedule"]["assignments"]::Vector{Any}

    if isempty(schedule)
        println("Baseline schedule has no assignments; cannot build what-if cases.")
        return
    end
    parsed = parse_input(input_dict)
    first, multi_course = pick_test_assignments(schedule)

    course_id = first["course_id"]
    week0     = Int(first["week"])          # already 0-indexed in your formatter
    day_str   = String(first["day"])
    p_start0  = Int(first["period_start"])
    room_id   = String(first["room_id"])

    println("\nUsing baseline assignment for basic tests:")
    println("  course_id    = $course_id")
    println("  week (0-based)= $week0")
    println("  day          = $day_str")
    println("  period_start = $p_start0")
    println("  room_id      = $room_id")

    # ----------------- 1. enforce_time_slot (should be trivially feasible) -----------------
    q1 = [Dict(
        "type" => "enforce_time_slot",
        "course_id" => course_id,
        "week" => week0,
        "day" => day_str,
        "period_start" => p_start0
    )]
    res1 = solve_what_if_query(input_dict, q1, f_star)
    print_what_if_summary("enforce_time_slot at baseline slot", res1)

    # ----------------- 2. veto_time_slot (may be feasible or infeasible) -----------------
    q2 = [Dict(
        "type" => "veto_time_slot",
        "course_id" => course_id,
        "week" => week0,
        "day" => day_str,
        "period_start" => p_start0
    )]
    res2 = solve_what_if_query(input_dict, q2, f_star)
    print_what_if_summary("veto_time_slot at baseline slot", res2)

    # ----------------- 3. veto_time_slot across all weeks -----------------
    q3 = [Dict(
        "type" => "veto_time_slot",
        "course_id" => course_id,
        "day" => day_str,
        "period_start" => p_start0
        # no "week" -> our encoder vetoes across all weeks
    )]
    res3 = solve_what_if_query(input_dict, q3, f_star)
    print_what_if_summary("veto_time_slot on all weeks at this day/period", res3)

    # ----------------- 4. veto_day (ban a course from a day it currently uses) ----------
    q4 = [Dict(
        "type" => "veto_day",
        "course_id" => course_id,
        "day" => day_str
    )]
    res4 = solve_what_if_query(input_dict, q4, f_star)
    print_what_if_summary("veto_day for a day used in baseline", res4)

    # ----------------- 5. enforce_room (force a course into baseline room) --------------
    q5 = [Dict(
        "type" => "enforce_room",
        "course_id" => course_id,
        "room_id" => room_id
    )]
    res5 = solve_what_if_query(input_dict, q5, f_star)
    print_what_if_summary("enforce_room to baseline room", res5)

    # ----------------- 6. enforce_before_time / after_time ------------------------------
    # For these, we try to construct constraints that baseline already satisfies,
    # so UDSP should be feasible with Δf ≈ 0, but that is not guaranteed (depends on model).

    # Find a somewhat-late period to use as "latest allowed"
    latest_period = p_start0 + Int(first["period_length"]) - 1
    # but your period_length is stored as number of periods, not in this dict; we’ll just
    # use p_start0+1 as a safe "ends by" bound
    latest_period = p_start0 + 1

    q6_before = [Dict(
        "type" => "enforce_before_time",
        "course_id" => course_id,
        "period_end" => latest_period
    )]
    res6 = solve_what_if_query(input_dict, q6_before, f_star)
    print_what_if_summary("enforce_before_time (course ends before/at a bound)", res6)

    # For after_time, ask that course starts at/after current start:
    q7_after = [Dict(
        "type" => "enforce_after_time",
        "course_id" => course_id,
        "period_start" => p_start0
    )]
    res7 = solve_what_if_query(input_dict, q7_after, f_star)
    print_what_if_summary("enforce_after_time (course starts after/at bound)", res7)

    # ----------------- 7. Multi-query conflict: enforce + veto same slot -----------------
    q8_conflict = [
        Dict(
            "type" => "enforce_time_slot",
            "course_id" => course_id,
            "week" => week0,
            "day" => day_str,
            "period_start" => p_start0
        ),
        Dict(
            "type" => "veto_time_slot",
            "course_id" => course_id,
            "week" => week0,
            "day" => day_str,
            "period_start" => p_start0
        )
    ]
    res8 = solve_what_if_query(input_dict, q8_conflict, f_star)
    print_what_if_summary("conflict: enforce and veto same slot", res8)

    # ----------------- 8. Optional: pick a multi-session course for day-level tests ------
    if multi_course !== nothing
        cid, assns = multi_course
        a1 = assns[1]
        day2 = String(a1["day"])
        println("\nUsing multi-session course $cid on day $day2 for extra tests...")

        q9 = [Dict(
            "type" => "veto_day",
            "course_id" => cid,
            "day" => day2
        )]
        res9 = solve_what_if_query(input_dict, q9, f_star)
        print_what_if_summary("veto_day for multi-session course", res9)
    else
        println("\n(No multi-session course found; skipping extra veto_day test.)")
    end

        # ----------------- 9. Availability-conflict test -----------------
    # Try to construct a scenario where the query contradicts instructor
    # availability C4: enforce a course on a day where that instructor
    # is completely unavailable.
    conflict_tuple = find_course_on_completely_unavailable_day(parsed)

    if conflict_tuple === nothing
        println("\n(No instructor-day with complete unavailability found; skipping availability-conflict test.)")
    else
        cid_conf, week0_conf, day_conf, p_start0_conf = conflict_tuple

        println("\nUsing availability-conflict test on:")
        println("  course_id     = $cid_conf")
        println("  week (0-based)= $week0_conf")
        println("  day           = $day_conf")
        println("  period_start  = $p_start0_conf (0-based)")

        # This is the "Prof only available on Friday, user enforces Wed" pattern:
        # CQ enforces the course into a day/time that violates C4, so UDSP should
        # be infeasible with an IIS that does NOT include minimality.
        q_avail_conflict = [Dict(
            "type" => "enforce_time_slot",
            "course_id" => cid_conf,
            "week" => week0_conf,
            "day" => day_conf,
            "period_start" => p_start0_conf
        )]

        res_avail = solve_what_if_query(input_dict, q_avail_conflict, f_star)
        print_what_if_summary(
            "availability conflict: enforce course on a day with no availability",
            res_avail
        )
    end

    println("\n================ WHAT-IF TESTS DONE ================\n")
end


run_what_if_tests

In [72]:
input_json = Dict(
    "term_config" => Dict(
        "num_weeks" => 4,
        "days" => ["Mon", "Tue", "Wed", "Thu"],
        "day_start_time" => "09:00",
        "day_end_time" => "15:00",
        "period_length_minutes" => 90
    ),

    "classrooms" => [
        Dict(
            "id" => "R101",
            "name" => "Room 101",
            "capacity" => 30
        ),
        Dict(
            "id" => "R201",
            "name" => "Room 201",
            "capacity" => 60
        ),
        Dict(
            "id" => "R301",
            "name" => "Room 301",
            "capacity" => 100
        )
    ],

    "instructors" => [
        Dict(
            "id" => "I1",
            "name" => "Prof. Alpha",
            "back_to_back_preference" => -1,
            "allow_lunch_teaching" => false
        ),
        Dict(
            "id" => "I2",
            "name" => "Prof. Beta",
            "back_to_back_preference" => 1,
            "allow_lunch_teaching" => true
        ),
        Dict(
            "id" => "I3",
            "name" => "Prof. Gamma",
            "back_to_back_preference" => 0,
            "allow_lunch_teaching" => false
        ),
        Dict(
            "id" => "I4",
            "name" => "Prof. Delta",
            "back_to_back_preference" => 1,
            "allow_lunch_teaching" => true
        )
    ],

    "courses" => [
        Dict(
            "id" => "CME101",
            "name" => "Linear Algebra for Engineers",
            "instructor_id" => "I1",
            "expected_enrollment" => 40,
            "type" => "full_term"
        ),
        Dict(
            "id" => "CME102",
            "name" => "Numerical Methods",
            "instructor_id" => "I1",
            "expected_enrollment" => 35,
            "type" => "full_term"
        ),
        Dict(
            "id" => "CME103",
            "name" => "Convex Optimization",
            "instructor_id" => "I2",
            "expected_enrollment" => 50,
            "type" => "full_term"
        ),
        Dict(
            "id" => "ECON201",
            "name" => "Econometrics",
            "instructor_id" => "I2",
            "expected_enrollment" => 45,
            "type" => "first_half_term"
        ),
        Dict(
            "id" => "CS110",
            "name" => "Introduction to Computer Systems",
            "instructor_id" => "I3",
            "expected_enrollment" => 80,
            "type" => "full_term"
        ),
        Dict(
            "id" => "STAT200",
            "name" => "Applied Statistics",
            "instructor_id" => "I4",
            "expected_enrollment" => 55,
            "type" => "second_half_term"
        )
    ],

    "students" => [
        Dict(
            "id" => "S01",
            "enrolled_course_ids" => ["CME101", "CME103", "CS110"]
        ),
        Dict(
            "id" => "S02",
            "enrolled_course_ids" => ["CME101", "CME102"]
        ),
        Dict(
            "id" => "S03",
            "enrolled_course_ids" => ["CME101", "CME103", "STAT200"]
        ),
        Dict(
            "id" => "S04",
            "enrolled_course_ids" => ["CME102", "CS110"]
        ),
        Dict(
            "id" => "S05",
            "enrolled_course_ids" => ["CME102", "CME103", "ECON201"]
        ),
        Dict(
            "id" => "S06",
            "enrolled_course_ids" => ["CME103", "CS110"]
        ),
        Dict(
            "id" => "S07",
            "enrolled_course_ids" => ["ECON201", "STAT200"]
        ),
        Dict(
            "id" => "S08",
            "enrolled_course_ids" => ["CME101", "ECON201", "CS110"]
        ),
        Dict(
            "id" => "S09",
            "enrolled_course_ids" => ["CME102", "STAT200"]
        ),
        Dict(
            "id" => "S10",
            "enrolled_course_ids" => ["CME101", "CS110", "STAT200"]
        ),
        Dict(
            "id" => "S11",
            "enrolled_course_ids" => ["CME103", "ECON201"]
        ),
        Dict(
            "id" => "S12",
            "enrolled_course_ids" => ["CS110", "STAT200"]
        ),
        Dict(
            "id" => "S13",
            "enrolled_course_ids" => ["CME101", "CME102", "ECON201"]
        ),
        Dict(
            "id" => "S14",
            "enrolled_course_ids" => ["CME103", "CS110", "STAT200"]
        ),
        Dict(
            "id" => "S15",
            "enrolled_course_ids" => ["CME102", "ECON201", "STAT200"]
        )
    ],

    "conflict_weights" => Dict(
        "global_student_conflict_weight" => 1.0,
        "instructor_compactness_weight" => 2.0,
        "preferred_time_slots_weight" => 0.5
    )
)

# now run the test harness
run_what_if_tests(input_json)



================ BASELINE SOLVE ================
📊 Julia solver started at 2025-12-03T18:29:55.498
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 6 courses, 4 weeks, 4 days, 4 periods
  Constraints: 4554
  Variables:   14720
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.05s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = -12.0
  S3_lunch_penalty     = 0.0
  Total objective      = -12.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
Instructor I1 pref = -1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Wed: T=2.0, edges(E)=T-1=1.0, B=1.0, contrib=-2.0
  Week 2, Day Wed: T=2.0, edges(E)=T-1=1.0, B=1.0, contrib=-2.0
  Week 3, Day Wed: T=2.0, edges(E)=T-1=1.0, B=1.0, contrib=-2.0
  Week 4, Day Wed: T=2.0, edges(E)=T-1=1.0, B=1.0, cont